<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/129_%E9%9D%9E%E5%90%8C%E6%9C%9FIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

非同期 I/O
==========

ソケット
--------

### ストリームソケット ###

**ソケット**（socket）は、プロセス間通信（IPC）の一種であるが、プラットフォームをまたぐ通信を可能とするという特徴を持つ。

プラットフォームをまたぐ通信には通信プロトコルを決めておく必要があり、TCP/IP がインターネットや企業ネットワークで事実上標準の通信プロトコルとして利用されている。

IP（Internet Protocol）は、`'100.50.200.5'` のような IP アドレスを用いて通信先の指定及び呼び出しを行うためのインターネット通信プロトコルである。IP アドレスには IPv4 と IPv6 が存在し記述の仕方が異なる。これらはアドレスファミリと呼ばれ、プログラム上ではそれぞれ `AF_INET`、 `AF_INET6` という定数で示される。

TCP（Transmission Control Protocol）は、「送ったデータが相手に届いたか、その都度確認しながら通信するやり方」のための通信プロトコルである。TCP では、クライアント・サーバー間の接続を確立してからデータの送受信を行う。「接続が確立する」とは、クライアントが通信を行うことをサーバーに通知し、サーバーがこれを承諾することをクライアントに通知し、さらにクライアントがサーバーの承諾を受け取ったことをサーバーに通知する、という 3 つのやり取りが全て行われることをいう。これを**スリーウェイハンドシェイク**という。データを送信するとき、**パケット**と呼ばれるデータを小分けに分割したもので送信し、相手からはパケットを受け取るたびに確認応答が来る。確認応答が来なければ再度同じパケットを送信する。TCP は、一見煩雑なやり方に見えるが、ネットワークが混み合って通信途中で通信内容が消滅してしまうことがあるので、実は一番確実で速い。

アプリケーション間では、扱うデータのフォーマットや手順を、HTTP、HTTPS（Web ブラウザ）、SMTP、POP3、IMAP4（電子メールソフト）、FTP（ファイル転送ソフト）といったプロトコルによって決めている。TCP/IP 通信を行うホストがどのアプリケーションにパケットを届けるかは、**ポート番号**によって決定される。

| プロトコル | ポート番号 |
|:--:|:--:|
| HTTP | 80	|
| HTTPS | 443 |
| SMTP | 25 |
| POP3 | 110 |
| IMAP4 | 143 |
| FTP | 20/21 |

ソケットは、アプリケーションの世界と TCP/IP の世界を結ぶ出入口（I/O）の API となるもので、多くのプラットフォームで実装されている。

とくに、UNIX 系 OS では、ソケットが同じホストでのプロセス間通信としても利用できて、**UNIX ドメインソケット**と呼ばれる。ソケットは無名のファイルのように扱われ、ファイルディスクリプタを指定することができる。ファイルをソケットとして扱うこともできる。

ソケットが対応する通信プロトコルは TCP に限られず、使用する通信プロトコルによってソケットタイプが区別される。TCP を使用したプロセス間通信を行うソケットは**ストリームソケット**という。ストリームソケットには、**クライアントソケット**と**サーバーソケット**がある。 Web ブラウザなどのクライアントアプリケーションは、クライアントソケットだけを使う。ウェブサーバーはサーバーソケットとクライアントソケットの両方を使う。

### ブロッキング I/O ###

標準ライブラリの `socket` モジュールは、ソケット API へのアクセスやネットワーク関連の便利な関数を提供している。

ネットワーク関連としては、たとえば `socket.gethostbyname()` 関数は、ホスト名を渡して呼び出すと、DNS サーバーから IPv4 アドレスを取得して返す。

In [ ]:
import socket
hostname = 'www.python.org'
socket.gethostbyname(hostname)

'146.75.28.223'

`socket.socket()` 関数は、ソケットの様々な機能をサポートするオブジェクト（ソケットオブジェクトと呼ぶ）を作成する。

``` python
socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)
```

| 引数 | 意味 |
|:---|:---|
| `family` | アドレスファミリを指定する。デフォルトはモジュール定数 `socket.AF_INET` であり、IPv4 が使用される。モジュール定数 `socket.AF_INET6` を指定すれ<br />ば、IPv6 が使用される。モジュール定数 `socket.AF_UNIX` を指定すれば、UNIX ドメインソケットとしてファイルが使用される |
| `type` | ソケットタイプを指定する。デフォルトはモジュール定数 `socket.SOCK_STREAM` であり、ストリームソケットとなる |

ソケットオブジェクトの主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `connect(address)` | `address` で示されるリモートソケットに接続する。アドレスファミリが `socket.AF_INET` や `socket.AF_INET6` の場合、<br />`address` は URL（または IP アドレス）とポート番号の 2 要素タプルとする | `None` |
| `send(bytes[, flags])` | ソケットにデータを送信する。ソケットはリモートソケットに接続済みでなければならない。戻り値として、送信したバイ<br />ト数を返す。全てのデータが送られたことを確認するには、戻り値を確認する必要がある | `int` |
| `sendall(bytes[, flags])` | `send()` との違いは、`bytes` の全データを送信するか、エラーが発生するまで処理を継続すること。正常終了の場合は<br /> `None` を返し、エラー発生時には例外が発生する | `None` |
| `recv(bufsize[, flags])` | ソケットからデータを受信し、受信したデータを表すバイト列を返す。一度に受信するデータの最大量は `bufsize` で指<br />定する。ハードウェアおよびネットワークの現実に最大限マッチするように、`bufsize` の値は比較的小さい 2 の累乗、<br />たとえば `4096`にすべき。空のバイト列が返された時はクライアントが切断されたことを示す | `bytes` |
| `close()` | ソケットを閉じる。通信相手から接続が切断される | `None` |

クライアントソケットは通信に先立って用意し、通信を終了するときに破棄する、というように使い捨てとなる。ガベージコレクション時に自動的に閉じられるのであるが、明示的に `close()` を呼び出すことが推奨されている。ソケットオブジェクトはコンテキストマネージャーとして使用でき、with 文の中から出るときに `close()` を呼ぶ。

次のコードの `request_get()` 関数は、クライアントソケットの使用例であり、サーバーに GET リクエストを行う。GET リクエストは、`urllib.request` モジュールや Requests パッケージを使えば簡単に書けるが、低水準なソケット API を使って `request_get()` 関数を書いている。

In [ ]:
import socket
import time

def request_get():
    response = b""
    with socket.socket() as sock:
        sock.connect(("example.com", 80))
        sock.sendall(b"GET / HTTP/1.0\r\nHost: example.com\r\n\r\n")
        bufsize = 4096
        recv = sock.recv(bufsize)
        while recv:
            response += recv
            recv = sock.recv(bufsize)
    return response

def main():
    print("計測開始")
    startall = time.time()
    for i in range(10):
        start = time.time()
        request_get()
        print("{} 処理時間: {:.3f} 秒".format(i, time.time() - start))
    print("{:.3f} 秒経過した".format(time.time() - startall))

if __name__ == "__main__":
    main()

計測開始
0 処理時間: 0.027 秒
1 処理時間: 0.027 秒
2 処理時間: 0.027 秒
3 処理時間: 0.026 秒
4 処理時間: 0.026 秒
5 処理時間: 0.026 秒
6 処理時間: 0.027 秒
7 処理時間: 0.027 秒
8 処理時間: 0.026 秒
9 処理時間: 0.026 秒
0.268 秒経過した


`request_get()` 関数の中では、まず `connect()` で WEB サーバーに接続する。接続が確立する前に次の `sendall()` が実行されればエラーとなるはずなので、`connect()` は WEB サーバーとの接続が確立するまで制御を返さない、つまり次の `sendall()` の実行をブロックしていることがわかる。また、WEB サーバーがリクエストを受け取っていなければ `recv()` がエラーとなるはずなので、`sendall()` は WEB サーバーから確認応答が来るまで制御を返さない、つまり次の `recv()` の実行をブロックしていることがわかる。このように I/O 操作が完了するまで他のタスクの開始をブロックする方式を**ブロッキング I/O**（blocking I/O）と呼ぶ。

`main()` 関数では、`request_get()` 関数を 10 回繰り返して処理時間を計測している。リクエストを行う個々のタスクにかかる時間の合計と、全体の経過時間がほぼ同じなので、各 I/O 操作が同期して処理されていることがわかる。この計測結果は人間にとっては十分に速いように思えるが、ナノ秒単位で動作する CPU の世界では非常に遅いと言える。頻繁にリクエストを行うアプリケーションでは、人間でも遅延を感じるようになる。遅延の原因は、ネットワークの通信速度やサーバーの処理能力などクライアントの CPU では解決できない要因による。したがって、シングルスレッドでのブロッキング I/O では処理速度を上げることは難しい。

なお、ここではクライアントソケットだけを使ったが、サーバーソケットの使い方については、公式ドキュメントの[ソケットプログラミング HOWTO](https://docs.python.org/ja/3/howto/sockets.html) あるいは以下の PyCon JP 2018 の講演配信動画（30:04）を参照。

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('L7j2zgtpV9c?start=1804', width=640, height=360)

### ノンブロッキング I/O ###

I/O 処理にかかる時間を改善する方法としてまず思いつくのは、マルチプロセスの利用である。しかしながら、 I/O ごとにプロセスを作成するという方法は、プロセス作成のコストが大きく、CPU コア数を上回る数のリクエストがある時にはコンテキストスイッチのコストも発生してしまう。

とくに、サーバープログラムでは、クライアントとの接続ごとに 1 プロセスを割り当てると、処理性能が著しく落ちてしまうことが知られている。この問題は **C10K 問題**（クライアント 1 万台問題）と呼ばれる。

Python では、GIL があっても、マルチスレッドで I/O バウンドな処理を効率よく実行することができた。マルチスレッドの利用ならばマルチプロセスよりは問題が軽減されるが、根本的な解決にはならない。大量のスレッドを作成すれば、やはりメモリ不足やコンテキストスイッチのコストの影響が出てしまうからである。OS がスレッドの数の上限を設定していることもある。また、マルチスレッドでは競合状態やデッドロックの問題が発生する可能性がある。

ここで、I/O 処理にかかる時間の内訳をみると、そのほとんどは I/O の準備が完了するまでの時間である。この時間中はホストの CPU は何もしていない。そこで、現代の OS は、シングルスレッドでも I/O 操作の完了を待たずに別の処理を開始することができるようにして I/O 処理の効率化を図るようになっている。このような処理方式を**ノンブロッキング I/O**（non-blocking I/O）と呼ぶ。

Python の場合、ソケットをノンブロッキング I/O にするには `socket.setblocking(False)` を使う。これは、ソケットを作成した後、使用する前に実行する。

次のコードでは、クライアントソケットの使用例をノンブロッキング I/O に変更している。

In [ ]:
import socket
import time

def request_get():
    response = b""
    with socket.socket() as sock:
        # ノンブロッキング
        sock.setblocking(False)
        # ノンブロッキングソケットは接続リクエストを送信すると、OSはエラーを起こすため
        try:
            sock.connect(("example.com", 80))
        except BlockingIOError:
            pass
        # ノンブロッキングソケットの接続がいつ確立されるか予測できないため、送信を繰り返す
        while True:
            try:
                sock.sendall(b"GET / HTTP/1.0\r\nHost: example.com\r\n\r\n")
                break
            except OSError:
                pass
        # レスポンスがいつ読み取れるか予測できないため、受信を繰り返す
        bufsize = 4096
        while True:
            try:
                recv = sock.recv(bufsize)
                while recv:
                    response += recv
                    recv = sock.recv(bufsize)
                break
            except OSError:
                pass
    return response

def main():
    print("計測開始")
    startall = time.time()
    for i in range(10):
        start = time.time()
        request_get()
        print("{} 処理時間: {:.3f} 秒".format(i, time.time() - start))
    print("{:.3f} 秒経過した".format(time.time() - startall))

if __name__ == "__main__":
    main()

計測開始
0 処理時間: 0.026 秒
1 処理時間: 0.026 秒
2 処理時間: 0.026 秒
3 処理時間: 0.026 秒
4 処理時間: 0.039 秒
5 処理時間: 0.025 秒
6 処理時間: 0.025 秒
7 処理時間: 0.025 秒
8 処理時間: 0.025 秒
9 処理時間: 0.025 秒
0.269 秒経過した


ノンブロッキング I/O としたソケットの `connect()` は接続の確立を待たずに制御を返す。 `sendall()` でデータを送信するためには接続済みでなければならないので、while ループで `sendall()` を試し続けている。`sendall()` もまた WEB サーバーからの確認応答を待たずに制御を返すため、while ループで `recv()` の呼び出しを繰り返している。

`main()` 関数で `request_get()` 関数を 10 回繰り返して処理時間を計測した結果は、ブロッキング I/O のときとほとんど変わっていない。`send()`、 `recv()`、 `connect()` がノンブロッキングとなっても、I/O 操作の完了を while ループで待っているため、結果的にブロッキングと変わらないことになっている。 while ループを使う限り、ノンブロッキング I/O は単にコードが複雑化しただけでメリットがないように見える。

### I/O 多重化とイベントループ ###

ノンブロッキング I/O の準備が完了したかどうかをアプリケーション側で監視する限り、処理効率がブロッキング I/O と変わらないので、OS 側でこれを監視する機能が提供されている。この機能を **I/O 多重化**（I/O multiplexing）という。具体的には次のとおり。

  * Unix 系 … select、epoll（Linux のみ）、kqueue（BSD のみ）などのシステムコール
  * Windows … I/O Completion Ports（IOCP）と呼ばれる API

これらは、複数のファイルディスクリプタを監視し、1 つ以上のファイルディスクリプタがある種の I/O 操作の準備が完了した状態（読み込み可能になった状態や、書き込み可能になった状態など）になるまで待機し、状態の変化を I/O イベントとしてアプリケーションに通知する。 select は多くのプラットフォームで利用可能であるが、監視対象のファイルディスクリプタを 1 つ 1 つ確認するアルゴリズムなので、ファイルディスクリプタの数 $n$ に対して計算量が $O(n)$ となる。epoll や kqueue はアルゴリズムが改善されて計算量が $O(1)$ つまり定数時間となるが、サポートしている OS が限られることに注意が必要である。なお、Windows では I/O 多重化はソケットに対してしか実装されていない（なのでファイルディスクリプタは単にソケットに読み替えること）。

I/O 多重化に対する標準の Python インターフェースとしては、 `select` モジュールと `selectors` モジュールがある。 `select` は上記のシステムコールに対するアクセスを提供していて、`selectors` モジュールは `select` の上に構築された高レベルなインターフェースを提供している。よって、通常は `selectors` を使う。

`selectors` では、 `events` という名前の属性や引数に、待機する I/O イベントがセットされる。これらはビットマスクであり、あらかじめ以下のモジュール定数が用意されている。

| 定数 | 意味 |
|:---|:---|
| `selectors.EVENT_READ` | 読み込み可能 |
| `selectors.EVENT_WRITE` | 書き込み可能 |

`selectors.SelectorKey` クラスは、I/O 多重化での監視対象と、待機する I/O イベントと、監視対象に関連付けられたコールバックを保持する。このコールバックは、通常、 I/O イベントが発生した後の処理を行うものである。

`selectors.SelectorKey` の属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `SelectorKey.fileobj` | 登録された監視対象 |
| `SelectorKey.fd` | 監視対象のファイルディスクリプタ |
| `SelectorKey.events` | 待機する I/O イベント |
| `SelectorKey.data` | 監視対象に関連付けられたコールバック |

`selectors.DefaultSelector` クラスは、I/O 多重化実装に対する統一的なインターフェースを提供し、現在のプラットフォームで利用できる最も効率的な I/O 多重化実装を使用する。たとえば、Linux では `select.epoll()` を使用する。主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `DefaultSelector.register(fileobj,`<br />` events, data=None)` | 監視対象 `fileobj`、待機する I/O イベント `events`、コールバック `data` の組み合わせを登録する。こ<br />れらに対応する新しい `SelectorKey` インスタンスを返す | `SelectorKey` |
| `DefaultSelector.unregister(fileobj)` | 監視対象 `fileobj` を登録から外す。`fileobj` に関連付けられた `SelectorKey` インスタンスを返す | `SelectorKey` |
| `DefaultSelector.select(timeout=None)` | 登録されたいくつかの監視対象が準備できたか、タイムアウトするまで待機し、`(key, events)` タプ<br />ルのリストを返す。`key` は準備が完了した監視対象に対応する `SelectorKey` インスタンスとなる。<br />`timeout` が `None`（デフォルト）なら監視対象が準備できるまで待機する | `list[tuple]` |
| `DefaultSelector.close()` | インスタンスを閉じる | `None` |

アプリケーションは、 `select()` を通じて I/O イベントの発生を OS から通知してもらうことになる。複数の I/O イベントを待つことになるので、while ループで `select()` を繰り返す必要がある。そのため、I/O 多重化での処理の流れは、以下のようになる。

  1. `register()` で監視対象を次々に登録していく。
  2. while ループの中で `select()` を呼び出す。`select()` は待機するが、監視対象が 1 つ以上準備完了となったら制御を返す。
  3. `select()` の戻り値から監視対象のソケットとコールバックを取得し、コールバックを実行する。2 に戻る。

手順 2 と手順 3 に着目すると、この while ループは、状態の変化というイベントの発生を待ち受け、どのソケットでどのイベントが発生したのかを知って、コールバックを呼び出す働きをする。このようなループを**イベントループ**（event loop）という。

次のコードの `RequestGet` クラスは I/O 多重化の使用例であり、`loop()` 関数の中でイベントループを実装している。関数ではなくクラスを定義した理由は、 `recv()` がイベントループで呼ばれるコールバックの中で実行されるため、関数ではサーバーのレスポンスを `main()` に返すことができないからである。サーバーのレスポンスは、 `RequestGet` の `response` 属性に格納される。

In [ ]:
from selectors import DefaultSelector, EVENT_WRITE, EVENT_READ
import socket
import time

_selector = DefaultSelector()
_stopping = False


class RequestGet:
    count: int

    def __init__(self, id):
        # サーバーのレスポンスを格納する属性
        self.response = b""
        # 処理時間計測用
        self._start = time.time()
        self._id = id

    def do(self):
        sock = socket.socket()
        # ノンブロッキング
        sock.setblocking(False)
        # ノンブロッキングソケットは接続リクエストを送信すると、OSはエラーを起こすため
        try:
            sock.connect(("example.com", 80))
        except BlockingIOError:
            pass
        _selector.register(sock, EVENT_WRITE, self.request)

    def request(self, sock, mask):
        _selector.unregister(sock)
        sock.sendall(b"GET / HTTP/1.0\r\nHost: example.com\r\n\r\n")
        _selector.register(sock, EVENT_READ, self.read)

    def read(self, sock, mask):
        global _stopping
        recv = sock.recv(4096)
        if recv:
            self.response += recv
        else:
            _selector.unregister(sock)
            sock.close()
            RequestGet.count -= 1
            if not RequestGet.count:
                _stopping = True
            print("{} 処理時間: {:.3f} 秒".format(self._id, time.time() - self._start))


def loop():
    # イベントループ
    while True:
        events = _selector.select(5.0)
        for key, mask in events:
            callback = key.data
            callback(key.fileobj, mask)
        if _stopping:
            break


def main():
    print("計測開始")
    startall = time.time()
    RequestGet.count = 10
    for i in range(RequestGet.count):
        rg = RequestGet(i)
        rg.do()
    loop()
    print("{:.3f} 秒経過した".format(time.time() - startall))


if __name__ == "__main__":
    main()

計測開始
1 処理時間: 0.024 秒
0 処理時間: 0.027 秒
3 処理時間: 0.024 秒
2 処理時間: 0.026 秒
4 処理時間: 0.025 秒
5 処理時間: 0.024 秒
6 処理時間: 0.024 秒
7 処理時間: 0.024 秒
9 処理時間: 0.025 秒
8 処理時間: 0.026 秒
0.034 秒経過した


`main()` 関数では、クラス変数 `RequestGet.count` の初期値（= 10）分の `RequestGet` インスタンスを作成し、それぞれ `do()` メソッドを呼び出す。各インスタンスは、タスクを完了した時に `RequestGet.count` 変数をデクリメントする。`RequestGet.count` が 0 になった時のインスタンスは `_stopping` を `True` にセットし、イベントループが停止する。

`RequestGet.do()` の中では、`RequestGet.request()` をコールバックに指定し、その `RequestGet.request()` の中でさらに `RequestGet.read()` をコールバックに指定するというように、コールバックをネストしている。コールバックをネストする理由は、クライアントソケットの `connect()` → `sendall()` → `recv()` という操作の順番を狂わせないようにするためである。

`main()` 関数で GET リクエストを 10 回繰り返す処理時間を計測した結果を見ると、全体の処理時間がほとんど 1 個の GET リクエストの処理時間と変わらないから、10 個の GET リクエストは同期処理ではなく非同期な並行処理が行われたことがわかる。 GET リクエストからサーバーのレスポンスを得るまでにかかる時間のほとんどは I/O の準備待ちの時間であり、I/O 多重化では同時に複数の I/O の準備待ちに入るため、全体の処理時間が 1 個の GET リクエストの処理時間とほとんど変わらなかったのである。

ノンブロッキング I/O は、実際に非同期処理されることがわかったので、これからは**非同期 I/O**（asynchronous I/O）と呼ぶことにする。これは Python の公式ドキュメント上の語法である（ノンブロッキング I/O と非同期 I/O を区別する流儀もあるので注意する）。

asyncio
-------

Python で非同期 I/O に基づくプログラムを記述できたが、いくつか不満もある。

  * コールバックのネストは可読性が低い。処理が複雑になるとネストが深くなってしまい、保守性も下がる。これを「コールバック地獄」と呼ぶ。
  * 低水準なソケット API を使っていて、`urllib.request` モジュールや Requests パッケージのような高水準なインターフェースが使えていない（`urllib.request` や Requests はブロッキング I/O に基づいている）。

1 つ目の不満を解消するため、非同期 I/O を実現するためのフレームワークとして標準ライブラリの `asyncio` パッケージが提供され、またこれを活用するために Python 言語に async/await 構文が導入された。

2 つ目の不満を解消するためには、非同期 I/O と `asyncio` に対応するサードパーティライブラリを使うしかない。

`asyncio` の 非同期 I/O API には、低レベル API と高レベル API の 2 種類がある。低レベル API は主にライブラリやフレームワークの開発者が利用するためのもので、それ以外の利用では高レベル API の使用が推奨されている。

### コルーチン ###

**コルーチン**（co-routine）は、 `asyncio` を使ったアプリケーションを書くのに推奨される方法とされる。 co-routine の意味は「協調的な」ルーチンである。ルーチンとは、サブルーチン（subroutine）の略称で、呼び出し可能なコードブロックを指し、呼び出し元に処理結果の値を返すものは関数（function）、返さないものはプロシージャ（procedure）と呼ばれる。これが「協調的」であるとは、自発的に処理を中断して呼び出し元に制御を戻すことを指している。コルーチンは、続きから処理を再開することができる。中断時の状態（ローカル変数と中断した位置）を保持し、再開時にその状態から処理を続行できる。

処理を中断・再開できるという性質は、ジェネレーターによく似ている。実際、コルーチンはジェネレーターをベースに開発されたので、以下の性質でもジェネレーターに似ている。

  * 関数のように定義できるが、通常の関数のように呼び出すと、実行されずオブジェクトが返される。
  * コルーチン関数を抜けるときは、`StopIteration` 例外を送出する。return 文で抜ける場合は、return 文に書いた式の値を `StopIteration` の `value` 属性に格納する。
  * コルーチンオブジェクトは、 `send()`、`throw()`、`close()` といったメソッドを持つ。

しかし、コルーチンは、以下の点でジェネレーターとは異なる。

  * コルーチン関数の中で `yield` や `yield from` を使用することはできない。
  * コルーチンオブジェクトは、 `__next__()` メソッドを持たないのでイテレーターではない。

コルーチンを定義するには async def 文を使用する。

In [ ]:
async def hello():
    print("Hello!")

hello()  # 通常の関数のように呼び出すと、実行されずオブジェクトが返される

<coroutine object hello at 0x7f90f82fc430>

コルーチンは `send()` メソッドで実行できる。

In [ ]:
try:
    hello().send(None)
except StopIteration as e:
    print("コルーチン関数を抜けるときに StopIteration 例外が発生した")
    assert e.value is None

Hello!
コルーチン関数を抜けるときに StopIteration 例外が発生した


### イベントループ ###

`asyncio` でもイベントループが使用され、コルーチンの実行の開始、再開はイベントループで制御される。 `asyncio` のイベントループは、単なる while ループではなくオブジェクトであり、内部属性 `_ready` に FIFO キュー（実体は `collections.deque()`）を保持する。内部キューにはコールバックが入り、これを「コールバックの実行がスケジュールされる」と表現する。イベントループは、内部キューから 1 つコールバックを取り出しては呼び出すことを while ループでひたすらに続ける。

ただし、`asyncio` ではイベントループは低レベル API とされ、アプリケーションの開発者が直接操作することはほとんどない。

以下は、イベントループに関するモジュール関数である。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `asyncio.get_running_loop()` | 現在実行中のイベントループを返す。実行中のイベントループがなければ `RuntimeError` 例外が発生する | イベントループ |
| `asyncio.get_event_loop()` | 現在のスレッドのイベントループを返す | イベントループ |
| `asyncio.set_event_loop(loop)` | `loop` を現在のスレッドのイベントループに設定する | `None` |
| `asyncio.new_event_loop()` | 新しいイベントループを作成して返す | イベントループ |

実は Colab はイベントループで動いている（JupyterLab も同様）。以下のコードで確認できる:

In [ ]:
import asyncio
asyncio.get_running_loop()

<_UnixSelectorEventLoop running=True closed=False debug=False>

`asyncio` のイベントループは、イベントループであるから、 I/O 多重化実装を使用して I/O イベントを待ち受けコールバックを呼び出す。 `get_event_loop()`、`set_event_loop()`、`new_event_loop()` では、現在のプラットフォームで利用できる最も効率的な I/O 多重化実装が使用されるように、 `asyncio.SelectorEventLoop` クラス（Unix 系 OS 用）と `asyncio.ProactorEventLoop` クラス（Windows 用）が使い分けられてイベントループが生成される。これらをユーザー独自のクラスに差し替えるように設定することも可能で、この設定はイベントループポリシーというオブジェクトで定義される。イベントループポリシーのクラスは、抽象基底クラスである `asyncio.AbstractEventLoopPolicy` を実装するサブクラスでなければならない。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `asyncio.get_event_loop_policy()` | プロセス全体にわたる現在のイベントループポリシーを返す | イベントループポリシー |
| `asyncio.set_event_loop_policy(policy)` | プロセス全体にわたる現在のイベントループポリシーを `policy` に設定する | `None` |

In [ ]:
import asyncio
asyncio.get_event_loop_policy()  # asyncio が提供するデフォルトのイベントループポリシーを返す

たとえば、サードパーティ製の [uvloop](https://uvloop.readthedocs.io/) パッケージが提供するイベントループポリシー `uvloop.EventLoopPolicy` を使用する場合は、次のように書く。

``` python
import asyncio
import uvloop
asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())
```

`uvloop.EventLoopPolicy` では、 libuv ライブラリで高速化したイベントループが使用されるようになる。ただし、libuv の制約から `uvloop` は Windows をサポートしない。

### Future ###

`asyncio` でも非同期処理の設計にフューチャーパターンが採用され、 `asyncio.Future` クラスとして実装されている。 `asyncio.Future` クラスは、非同期処理の未来の結果を表現する。イベントループと接続し、スレッドセーフではなない。イベントループは、自身に接続した `Future` オブジェクトを作成することができる。

ただし、 `asyncio` では `Future` クラスそのものは低レベル API とされる。通常、アプリケーション水準のコードで `Future` オブジェクトを作成する必要はない。

`Future` は、完了・未完了・キャンセルといった状態を持つ。具体的には、内部属性 `_state` に以下の文字列定数がセットされる。

| 定数 | 意味 |
|:---|:---|
| `'PENDING'` | 完了していない状態 |
| `'CANCELLED'` | キャンセルされた状態 |
| `'FINISHED'` | 完了した状態。オブジェクトに結果か例外がセットされている |

通常は、内部属性 `_state` を直接操作するのではなく、以下のメソッドを使用する。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `future.done()` | オブジェクトが完了しているなら `True` を返す | `bool` |
| `future.cancelled()` | オブジェクトがキャンセルされているなら `True` を返す | `bool` |
| `future.result()` | オブジェクトにセットされた結果を返す。オブジェクトに例外がセットされた場合は、その例外を送出する。オブジェクトがキャン<br />セルされた場合は `asyncio.CancelledError` 例外を送出する。オブジェクトが未完了の場合は `asyncio.InvalidStateError` <br />例外を送出する | 結果 |
| `future.exception()` | オブジェクトにセットされた例外を返す。例外がセットされていないときは `None` を返す。オブジェクトがキャンセルされた場合<br />と未完了の場合は `result()` と同じ | 例外 &#124; `None` |

また、`Future` は、内部属性 `_callbacks` に、そのオブジェクトのみを引数として取るようなコールバックのリストを保持する。 `Future` オブジェクトを完了とマークし結果をセットする時、同時に `_callbacks` にある全てのコールバックの実行がスケジュールされるため、 `_callbacks` の中のコールバックは**完了時コールバック**と呼ばれる。完了時コールバックは、 `asyncio` で並行処理を実現するための仕組みであるが、通常、アプリケーション水準のコードで直接操作されるものではない。

### Task ###

イベントループとコールバックの存在は、高レベル API として提供される `asyncio.Task` クラスとモジュール関数によって完全に隠ぺいされる。

  * `Task` クラスは、 `asyncio.Future` クラスのサブクラスであり、イベントループと接続する。
  * `Task` クラスは、コルーチンをラップする（`_coro` 属性に格納する）。
  * `Task` クラスのイニシャライザ（`Task.__init__()`）によって `Task.__step()` の実行が引数なしでスケジュールされる。

内部メソッド `Task.__step()` は、ラップしたコルーチン `_coro` の実行と例外処理を行う。

  1. `Task.__step()` が引数なしで呼び出された場合、 `_coro.send(None)` を実行する。
  2. `Task.__step(exc)` のように引数付きで呼び出された場合、 `exc` は例外インスタンスであり、 `_coro.throw(exc)` を実行する。コルーチン内で `exc` 例外が発生し、例外処理をしない場合、もしくは違う例外を発生させるなら、その例外は呼び出し元である `Task.__step()` へ伝搬される。
  3. `Task.__step()` が `StopIteration` 例外を捕捉した場合、オブジェクトを完了とマークし、コルーチンの結果（`StopIteration` の `value` 属性値）をセットする。
  4. `Task.__step()` が `StopIteration` 以外の例外を捕捉した場合、オブジェクトを完了とマークし、例外をセットする。

つまり、**`Task` オブジェクトを作成すると、自動的にイベントループの次の回でコルーチンの実行が開始され、 `Task` オブジェクトの `result()` メソッドを呼び出してコルーチンの結果を取得できる。ただし、コルーチン側で発生した例外を処理していない場合に、 `result()` メソッドはその例外を送出する**。

``` python
asyncio.run(main, *, debug=None, loop_factory=None)
```

この関数は、処理の起点となるコルーチン `main` を実行し、その結果を返すか、例外を発生させる。以下の処理が暗黙のうちに行われる。

  1. イベントループを作成する。
  2. `main` コルーチンをラップする `Task` オブジェクトを作成する。
  3. `Task.__step()`（2 で作成された `Task` オブジェクトのメソッド）が引数なしでスケジュールされる。
  4. イベントループを実行する。スケジュールされていた `Task.__step()` が引数なしで呼び出される。
  5. `Task` オブジェクトが完了すると、イベントループを停止し、その `result()` メソッドを呼び出し戻り値を返す。ただし、`Task` オブジェクトに例外がセットされていると、`result()` メソッドはその例外を送出する。
  6. イベントループを閉じる。

| 引数 | 意味 |
|:---|:---|
| `main` | コルーチンの識別子ではなく、コルーチンオブジェクトを指定する必要がある |
| `debug` | キーワード専用。`True` の場合、イベントループはデバッグモードで実行される |
| `loop_factory` | キーワード専用。このオプションはイベントループの作成に使用される。`None` の場合、` asyncio.new_event_loop()` が使用される<br />（Python 3.12 で追加） |

`asyncio` はイベントループのネストを許さない。 `asyncio.run()` を呼び出した時に別の実行中のイベントループが存在している場合、 `RuntimeError` 例外が発生する。Colab はイベントループで動いているので、<font color="red">デフォルトでは Colab 上で `asyncio.run()` を使用することはできない</font>。

サードパーティ製の [nest_asyncio](https://pypi.org/project/nest-asyncio/) モジュールは、 `asyncio.run()` を置き換えてイベントループのネストを可能にする `nest_asyncio.apply()` 関数を提供する。ライセンスは BSD 2-clause License。Colab ではデフォルトで `nest_asyncio` がインストールされている。使い方は、次の 2 行をプログラムの最初に挿入するだけである。

``` python
import nest_asyncio
nest_asyncio.apply()
```

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def hello():
    print("Hello!")

asyncio.run(hello())

Hello!


複数の処理の起点となるコルーチンを実行する場合、たとえば:

``` python
asyncio.run(coro1())
asyncio.run(coro2())
```

この方法では、コルーチンごとに新しいイベントループを作成して閉じることになるため、コストがかかる。これは、処理の起点となるコルーチンが非常に多数ある場合に問題になる可能性がある。この問題を回避するため、Python 3.11 で `asyncio.Runner` クラスが追加された。 `asyncio.Runner` を使用すると、複数のコルーチンの非同期実行をスマートに記述できる。

``` python
asyncio.Runner(*, debug=None, loop_factory=None)
```

このクラスのインスタンス化では、イベントループが 1 つだけ作成される。インスタンスメソッドは以下のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `Runner.run(main, *, context=None)` | 内部のイベントループでコルーチン `main` を実行する。コルーチンの結果を返すか、例外を発生させる | 結果 |
| `Runner.close()` | オブジェクトを閉じる。オブジェクトを閉じると内部のイベントループも閉じられる | `None` |
| `Runner.get_loop()` | 内部のイベントループを返す | イベントループ |

`asyncio.Runner` はコンテキストマネージャーとして使用でき、 `close()` メソッドは with ブロックを終了するときに呼び出される。

たとえば、1 つのコルーチンを実行する関数、つまり `asyncio.run()` と等価な関数は次のように書ける:

``` python
def run(main, *, debug=None, loop_factory=None):
    with Runner(debug=debug, loop_factory=loop_factory) as runner:
        return runner.run(main)
```

複数のコルーチンを実行するには、 with ブロックに `Runner.run()` 呼び出しの行を追加するだけである。たとえば

``` python
with asyncio.Runner() as runner:
	runner.run(coro1())
	runner.run(coro2())
```

``` python
asyncio.create_task(coro, *, name=None, context=None)
```

この関数は、`coro` コルーチンをラップする `Task` オブジェクトを作成して返す。`Task` オブジェクトは現在実行中のイベントループに接続するので、現在実行中のイベントループがない場合にこの関数を呼び出すと `RuntimeError` 例外が発生する。

| 引数 | 意味 |
|:---|:---|
| `coro` | コルーチンの識別子ではなく、コルーチンオブジェクトを指定する必要がある |
| `name` | キーワード専用。`Task` オブジェクトの名前を定義する。省略した場合、デフォルトの名前が生成される。`get_name()` メソッドで名前にアクセスできる |

``` python
asyncio.current_task(loop=None)
```

この関数は、現在実行中の `Task` オブジェクトを返すが、なければ `None` を返す。`loop` が `None` の場合、`asyncio.get_running_loop()` が現在のイベントループを取得するのに使われる。

``` python
asyncio.all_tasks(loop=None)
```

イベントループ `loop` のまだ完了していないすべてのタスクの集合を返す。`loop` が `None` の場合、 `asyncio.get_running_loop()` が現在のイベントループを取得するのに使われる。

### awaitable ###

`__await__()` メソッドを持つオブジェクトのことを、 **awaitable オブジェクト（待機可能オブジェクト）**という。ただし、 `__await__()` メソッドは、イテレーターを返す必要がある。以下のオブジェクトは awaitable である。

  * コルーチン
  * `Future` およびサブクラス `Task` のインスタンス（`Task` のインスタンスの場合、 `Future.__await__()` を継承する）

コルーチン関数の中では、ジェネレーター関数での yield from 式に相当する await 式が使える。

``` python
await <awaitable>
```

このように、 await 式は awaitable オブジェクトを伴う。普通の関数（たとえば `time.sleep()` 関数）を await 式に置くことはできない。

await 式は式なので、 await 式の評価結果を `=` で変数に代入したり、 return 文で返すことができる。なお、 await 式の結合は添字表記（`[]`）と属性参照（`.`）以外のどの演算子よりも高いので、たとえば `await foo()**2` は `(await foo())**2` と評価される。

await 式の評価の際、暗黙的に yield from への変換が行われ、 `__await__()` が返すイテレーターへの委譲が行われる。 await のチェーンを辿っていくと yield する関数、すなわちジェネレーター関数に行き着く場合、その await のチェーンの処理はジェネレーターのチェーンと同様となる。すなわち、呼び出し元のコルーチンの実行は中断し（制御を呼び出し元に戻して）、ジェネレーターイテレーターが尽きるまで待機することになる。

コルーチンの `__await__()` メソッドは、コルーチンをラップしたジェネレーターイテレーターを返す。そもそもコルーチン関数の中には yield を使用できないので、コルーチンを伴う await 式の場所では呼び出し元のコルーチンの実行（`send()` メソッドによる実行）は中断せず最後まで実行される。実際、次のコードでこの事実を確認できる:

In [ ]:
async def hello():
    print("Hello!")

async def main():
    await hello()
    await hello()
    return "end"

it = main()
try:
    it.send(None)  # 1回の send() で main() が最初の await 式で中断せず return 文まで実行される
except StopIteration as e:
    print(e.value)

Hello!
Hello!
end


処理の起点となるコルーチンの実行（`send()` メソッドによる実行）が await 式で中断するのは、次の場合である。

  1. **`types` モジュールが提供する `@types.coroutine` でデコレートされたジェネレーター関数を await 式に置く場合**  
`@types.coroutine` でデコレートされたジェネレーター関数は、コルーチン関数に変換される（`__await__()` メソッドを実装する必要はない）。
  2. **`Future` または `Task` のインスタンスを await 式に置く場合**  
`Future.__await__()` は、ジェネレーター関数であり、未完了ならオブジェクト自身を yield し、完了しているなら `Future.result()` を return する。
  3. **コルーチン版の `sleep()` である `asyncio.sleep()` を await 式に置く場合**  
`asyncio.sleep()` の内部では `Future` オブジェクトを作成しこれを伴う await 式を return しているので、本質的には 2 の場合と同じ。

1 の方法は、一種の裏技と言えるものであり、一般のアプリケーション開発で使用するものではない。 2 の方法は、複数のコルーチンを並行処理することができ、 await 式の中心的な使い方である。 3 の方法は、実用的な意味はあまりないが、コルーチンをチェーンする例を示す場合に使われる。

``` python
asyncio.sleep(delay, result=None)
```

このコルーチンが実行されると、`delay` 秒だけ停止する。 `result` は、コルーチン完了時にそれが呼び出し元に返される（＝ `StopIteration` 例外の `value` 属性に `result` を持たせる）。

次は、コルーチンをチェーンする例である:

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    result = await compute(x, y)
    print("%s + %s = %s" % (x, y, result))

asyncio.run(print_sum(1, 2))

Compute 1 + 2 ...
1 + 2 = 3


処理の起点となる `print_sum()` の中では、 `compute()` → `asyncio.sleep()`（→ `Future` オブジェクト）という await のチェーンがある。

この例のシーケンス図（[公式ドキュメント](https://docs.python.org/ja/3.6/library/asyncio-task.html)より引用）:

![](https://docs.python.org/ja/3.6/_images/tulip_coro.png)

### Task による並行処理 ###

`Task` オブジェクトを伴う await 式を複数使用して、コルーチンを並行処理することができる。その仕組みは以下のとおり。

まず、 処理の起点となるコルーチンを実行する `Task.__step()` は、 1 つの `Task` オブジェクトを伴う await 式を以下のように処理する。

  1. **コルーチンの実行の中断**:  
`Task.__step()` は、 `_coro.send(None)`（`_coro` はラップしたコルーチン）を実行し、これにより await 式に置かれた `Task` オブジェクトの `__await__()` への委譲が行われる。 `__await__()` は、オブジェクトが未完了ならオブジェクト自身を yield する。 `Task.__step()` は、 yield された `Task` オブジェクトの完了時コールバックリストに `Task.__wakeup()`（処理の起点となるコルーチンを実行している方の `Task` のメソッド）を追加して終了する。
  2. **完了時コールバックの呼び出し:**  
await 式の `Task` オブジェクトが完了した時（つまり結果か例外がセットされた時）、完了時コールバックリストの仕組みにより `Task.__wakeup()` の実行がスケジュールされる。イベントループの次の回で、 `Task.__wakeup()` が引数として await 式の `Task` オブジェクトを渡されて呼び出される。
  3. **コルーチンの実行の再開:**  
`Task.__wakeup()` は、 await 式の `Task` オブジェクトの `result()` メソッドを呼び出す。
      * await 式の `Task` オブジェクトが例外をセットされた場合、`result()` がその例外を送出するので、 `Task.__wakeup()` はこれを捕捉し `Task.__step(exc)` を呼び出す（`exc` は捕捉した例外インスタンス）──`_coro.throw(exc)` が実行される。
      * await 式の `Task` オブジェクトが結果をセットされた場合、 `Task.__wakeup()` は `Task.__step()` を引数なしで呼び出す。  `_coro.send(None)` が実行され、 await 式の値がその結果で評価される（`__await__()` が送出する `StopIteration` 例外の `value` 属性が使われる）。

複数の await 式を使用する場合、 await 式に置く `Task` オブジェクトの作成時には、ラップされたコルーチンの実行がスケジュールされることに注意する。 I/O の準備待ちなどで制御が呼び出し元のコルーチンにすぐ戻ってくる場合、 `asyncio.create_task()` を並べれば、それぞれラップされたコルーチンの実行がほぼ同時に始まる。完了時コールバックの仕組みによって、効率よく await 式の評価が行われる。これが、複数のコルーチンが並行処理される仕組みである。

次のコードは、 `compute()` コルーチンから複数の `Task` オブジェクトを作成しておき、その後に await 式を並べた例である:

In [ ]:
import asyncio
import time

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def compute(x, y):
    print("Compute {} + {} ... [at {}]".format(x, y, time.strftime("%X")))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    print("started at {}".format(time.strftime("%X")))
    task1 = asyncio.create_task(compute(x, y))
    task2 = asyncio.create_task(compute(x * 2, y * 2))
    task3 = asyncio.create_task(compute(x * 10, y * 10))
    result1 = await task1
    result2 = await task2
    result3 = await task3
    print("{} + {} = {}".format(x, y, result1))
    print("{} + {} = {}".format(x * 2, y * 2, result2))
    print("{} + {} = {}".format(x * 10, y * 10, result3))
    print("finished at {}".format(time.strftime("%X")))

asyncio.run(print_sum(1, 2))

started at 00:35:00
Compute 1 + 2 ... [at 00:35:00]
Compute 2 + 4 ... [at 00:35:00]
Compute 10 + 20 ... [at 00:35:00]
1 + 2 = 3
2 + 4 = 6
10 + 20 = 30
finished at 00:35:01


各 `Task` オブジェクトでラップされた `compute()` コルーチンはそれぞれ 1 秒待機するが、全体の処理が 1 秒で終了している。 `asyncio.create_task()` の呼び出しで `compute()` コルーチンが実行されるが、 `asyncio.sleep()` が制御をすぐに `print_sum()` に戻すので、 3 つの `asyncio.create_task()` がほぼ同時に呼び出され、それぞれにラップされた `compute()` コルーチンもほぼ同時に実行される。最初の await 式で `Task` オブジェクトが完了するまで待機し、その結果を取得する時には、他の `Task` オブジェクトも完了している。このため、以降の await 式ではほとんど待機することなく `Task` オブジェクトの結果を取得するのである。

複数の `asyncio.create_task()` 呼び出しを 1 つの場所にまとめず、間に await 式を置くと、await 式で `Task` オブジェクトの完了を待機するので、次の `asyncio.create_task()` がブロックされ、結局、複数のコルーチンが同期処理されることになる。たとえば、上記のコードで `asyncio.create_task()` と await 式を次のように並べる:

``` python
    task1 = asyncio.create_task(compute(x, y))
    result1 = await task1
    task2 = asyncio.create_task(compute(x * 2, y * 2))
    result2 = await task2
    task3 = asyncio.create_task(compute(x * 10, y * 10))
    result3 = await task3
```

このように並べると、`task1 = asyncio.create_task(compute(x, y))` で `compute(x, y)` が開始し、`await task1` で `task1` の完了まで待機し、完了したら今度は `task2 = asyncio.create_task(compute(x * 2, y * 2))` で `compute(x * 2, y * 2)` が開始するというように、複数のコルーチンが同期処理されることになる。変数を使わない以下のコードも等価である:

``` python
    result1 = await asyncio.create_task(compute(x, y))
    result2 = await asyncio.create_task(compute(x * 2, y * 2))
    result3 = await asyncio.create_task(compute(x * 10, y * 10))
```

In [ ]:
import asyncio
import time

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def compute(x, y):
    print("Compute {} + {} ... [at {}]".format(x, y, time.strftime("%X")))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    print("started at {}".format(time.strftime("%X")))
    result1 = await asyncio.create_task(compute(x, y))
    result2 = await asyncio.create_task(compute(x * 2, y * 2))
    result3 = await asyncio.create_task(compute(x * 10, y * 10))
    print("{} + {} = {}".format(x, y, result1))
    print("{} + {} = {}".format(x * 2, y * 2, result2))
    print("{} + {} = {}".format(x * 10, y * 10, result3))
    print("finished at {}".format(time.strftime("%X")))

asyncio.run(print_sum(1, 2))

started at 00:36:51
Compute 1 + 2 ... [at 00:36:51]
Compute 2 + 4 ... [at 00:36:52]
Compute 10 + 20 ... [at 00:36:53]
1 + 2 = 3
2 + 4 = 6
10 + 20 = 30
finished at 00:36:54


以下の関数を使用すると、複数の `Task` オブジェクトによる並行処理の書き方に注意を払う必要がなくなる。

``` python
asyncio.wait(fs, *, timeout=None, return_when=ALL_COMPLETED)
```

これは、コルーチン関数であり、`fs` イテラブルの要素である `Task` オブジェクトを同時に実行し、`return_when` で指定された条件が満たされるまでブロックする。「完了した `Task` オブジェクトの集合」と「未完了の `Task` オブジェクトの集合」の 2 要素タプルを返す。

| 引数 | 意味 |
|:---|:---|
| `fs` | `Task` オブジェクトを要素とするイテラブル。空であってはならない |
| `timeout` | 浮動小数点数または整数が指定されていたら、`timeout` 秒でタイムアウトする。`TimeoutError` 例外は発生しない |
| `return_when` | この関数がいつ結果を返すのかを表す以下の定数のどれか 1 つを指定できる<br /><br />・`asyncio.FIRST_COMPLETED`: いずれかの `Task` オブジェクトが完了したかキャンセルされたときに返す<br /><br />・`asyncio.FIRST_EXCEPTION`: `Task` オブジェクトで例外が発生すると戻る。`Task` オブジェクトで例外が発生しない場合、`ALL_COMPLETED` と同等に<br />　なる<br /><br />・`asyncio.ALL_COMPLETED`: すべての `Task` オブジェクトが完了したかキャンセルされたときに返す（デフォルト） |

`asyncio.wait()` が返す「完了した `Task` オブジェクトの集合」と「未完了の `Task` オブジェクトの集合」は集合なので、渡した `fs` の順番は保たれない。

次のコードは、`asyncio.wait()` の使用例である:

In [ ]:
import asyncio
import time

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def compute(x, y):
    print("Compute {} + {} ... [at {}]".format(x, y, time.strftime("%X")))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    print("started at {}".format(time.strftime("%X")))
    tasks = set()
    tasks.add(asyncio.create_task(compute(x, y)))
    tasks.add(asyncio.create_task(compute(x * 2, y * 2)))
    tasks.add(asyncio.create_task(compute(x * 10, y * 10)))
    done, pending = await asyncio.wait(tasks)
    if len(done) == 3:
        print('All done')
    print("finished at {}".format(time.strftime("%X")))

asyncio.run(print_sum(1, 2))

started at 00:39:40
Compute 1 + 2 ... [at 00:39:40]
Compute 2 + 4 ... [at 00:39:40]
Compute 10 + 20 ... [at 00:39:40]
All done
finished at 00:39:41


`asyncio.wait()` による並行処理で例外処理を行うには、普通の try-except で囲い例外を捕捉する。 `return_when` 引数が `asyncio.ALL_COMPLETED`（デフォルト）の場合、ある `Task` オブジェクトでエラーが発生しても戻ることはなく、全ての `Task` オブジェクトが結果か例外をセットされ完了するまでブロックされ、発生したエラーはすべて例外として送出される。 `return_when` 引数が他の定数に設定された場合、エラーの情報は捨てられる。

次のコードでは、`foo()` コルーチンで `ZeroDivisionError` 例外と `TypeError` 例外が発生するような引数を渡している:

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    try:
        tasks = [asyncio.create_task(foo(x)) for x in (11, 0, 12, "13", 14)]
        done, pending = await asyncio.wait(tasks)
        print(f"{len(done)=}")  # asyncio.wait() 中にエラーが発生しても実行される
    except Exception as err:
        print(f"{type(err).__name__}: {err}")

asyncio.run(main())

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-21' coro=<foo() done, defined at <ipython-input-4-3952e14288f3>:7> exception=ZeroDivisionError('division by zero')>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "<ipython-input-4-3952e14288f3>", line 8, in foo
    await asyncio.sleep(1 / x)
ZeroDivisionError: division by zero
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-23' coro=<foo() done, defined at <ipython-input-4-3952e14288f3>:7> exception=TypeError("unsupported operand type(s) for /: 'int' and 'str'")>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "<ipython-input-4-3952e14288f3>", line 8, in foo
    await asyncio.sleep(1 / x)
TypeError: unsupported operand type(s) for /: 'int' and 'str'


len(done)=5


``` python
asyncio.gather(*coros_or_futures, return_exceptions=False)
```

この関数は、可変長位置引数として与えられたコルーチンまたは `Task` オブジェクトを並行処理し、それらの結果を可変長位置引数で渡した順番どおりにリストにして、そのリストで結果がセットされる `Future` オブジェクトを返す。

| 引数 | 意味 |
|:---|:---|
| `*coros_or_futures` | コルーチンまたは `Task` オブジェクト。コルーチンである場合、自動的に `Task` としてスケジュールされる |
| `return_exceptions` | `False`（デフォルト）の場合、最初の例外を送出する。`True` の場合、すべての `Task` オブジェクトの完了を待ち、例外を含む結果をリストで<br />返す |

コルーチンまたは `Task` オブジェクトの結果を取得したい場合は、 `asyncio.wait()` より `asyncio.gather()` の方が便利である。

次のコードは、`asyncio.gather()` の使用例である:

In [ ]:
import asyncio
import time

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def compute(x, y):
    print("Compute {} + {} ... [at {}]".format(x, y, time.strftime("%X")))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    print("started at {}".format(time.strftime("%X")))
    results = await asyncio.gather(
        compute(x, y),
        compute(x * 2, y * 2),
        compute(x * 10, y * 10),
    )
    print("{} + {} = {}".format(x, y, results[0]))
    print("{} + {} = {}".format(x * 2, y * 2, results[1]))
    print("{} + {} = {}".format(x * 10, y * 10, results[2]))
    print("finished at {}".format(time.strftime("%X")))

asyncio.run(print_sum(1, 2))

started at 00:42:30
Compute 1 + 2 ... [at 00:42:30]
Compute 2 + 4 ... [at 00:42:30]
Compute 10 + 20 ... [at 00:42:30]
1 + 2 = 3
2 + 4 = 6
10 + 20 = 30
finished at 00:42:31


`asyncio.gather()` による並行処理で例外が発生する場合、オプション `return_exceptions` の設定によって例外処理の方法が異なる。デフォルトの `False` の場合、普通の try-except で囲い例外を捕捉する。どれか 1 つでもエラーが発生した場合、`asyncio.gather()` は最初のエラーを例外として送出して、呼び出し元に戻る。他の `Task` オブジェクトでのエラーに関する情報は得られない:

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    try:
        tasks = [foo(x) for x in (11, 0, 12, '13', 14)]
        results = await asyncio.gather(*tasks)
        print(results)
    except Exception as err:
        print(f"{type(err).__name__}: {err}")

asyncio.run(main())

ZeroDivisionError: division by zero


このコードを実行すると、最初の例外の `ZeroDivisionError` 例外のみ捕捉することができ、`TypeError` 例外は捕捉できない。また、例外が発生していない `Task` オブジェクトの結果は返ってこない。

一方、 `return_exceptions=True` とする場合、例外は送出されないので、 try-except で囲う必要はない:

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    tasks = [foo(x) for x in (11, 0, 12, '13', 14)]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    print(f"{results=}")

asyncio.run(main())

results=[11, ZeroDivisionError('division by zero'), 12, TypeError("unsupported operand type(s) for /: 'int' and 'str'"), 14]


`ZeroDivisionError` 例外も `TypeError` 例外も送出されず、例外オブジェクトが戻り値のように結果リストに追加される。また、例外が発生していない `Task` オブジェクトの結果も全て返ってくる。この場合、例外を処理するには、戻り値のリストをチェックする必要がある。

`return_exceptions=True` は便利そうに見えるが、同じようなタスクを多数同時に実行する場合は無駄が多い可能性がある。全ての `Task` オブジェクトで同じ例外が発生するような場合（たとえば接続エラーが発生する場合）でも、 `return_exceptions=True` とした `asyncio.gather()` は、全ての `Task` オブジェクトが例外をセットする形で完了するまで待つ（`asyncio.wait()` のデフォルト動作も同様の問題を抱える）。

### Task のキャンセル ###

`asyncio.create_task()` から返された `Task` オブジェクトを保持することにより、ラップしたコルーチンの実行を完了前にキャンセルすることができる。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `Task.cancel(msg=None)` | 自身のキャンセルを要求する。`msg` は `asyncio.CancelledError` のインスタンス化の際に渡される。オブジェクトがすでに完了<br />していた場合、このメソッドは `False` を返すこと以外は何もしない。そうでない場合、このメソッドは `True` を返す | `bool` |

未完了の `Task` オブジェクト（処理の起点となるコルーチンを実行している `Task` と区別するため以下小文字で task と表す）に対して、その `cancel()` メソッドを呼び出した場合、イベントループの次の回で、 `_coro.throw(asyncio.CancelledError(msg))` が実行されるようになる（`_coro` はラップされたコルーチン）。 `Task.__step()` は、 `_coro` が送出した `CancelledError` 例外を捕捉すると、 task をキャンセルとマークする。このような処理が行われるため、 **`cancel()` を呼び出してから task がキャンセルされるまでにタイムラグがある**。

コルーチンでは、task がキャンセルされた場合の処理をカスタマイズする目的で `CancelledError` 例外を捕捉することができる。この場合、 `CancelledError` 例外を再度送出しなければならない。そうしないと、`Task.__step()` は `CancelledError` を捕捉しないから task をキャンセルとマークしない。

`CancelledError` は `Exception` ではなく `BaseException` の直接のサブクラスになっている。このため、一般的な例外を全て抑制して `CancelledError` 例外だけ再送出するように try-except をコード化することは容易である。

`asyncio.wait()` や `asyncio.gather()` で例外が送出された場合、エラーが発生していない `Task` オブジェクトではコルーチンの処理が続いている場合がある。このように `Task` オブジェクトが残ってしまうと、I/O 操作ではソケットを閉じることができない可能性がある。そこで、未完了の `Task` オブジェクトをキャンセルする必要がある。

次のコードでは、処理時間の異なる複数の `Task` オブジェクトを作成し、例外を発生させている:

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()


async def foo(x):
    try:
        await asyncio.sleep(1 / x)
        return x
    except asyncio.CancelledError:
        print("キャンセルされた場合の処理を実行中...")
        raise  # 再送出する


async def main():
    try:
        tasks = [
            asyncio.create_task(foo(0.1), name="long task"),  # 10 秒かかるタスク
            asyncio.create_task(foo(0), name="wrong task"),  # ZeroDivisionError が発生するタスク
            asyncio.create_task(foo(10), name="normal task"),  # 0.1 秒かかるタスク
        ]
        results = await asyncio.gather(*tasks)
        print(f"{results=}")  # 例外が発生するので実行されない
    except Exception as err:
        print(f"{type(err).__name__}: {err}")

    # normal task が完了する頃合いまで待機
    await asyncio.sleep(0.1)

    # main() コルーチンを動かしている Task を除いて未完了な Task を集める
    pendings = [t.get_name() for t in asyncio.all_tasks() if t != asyncio.current_task()]
    print(f"{pendings=}")  # long task が未完了であることがわかる

    # 未完了な Task をキャンセル
    print("未完了な Task のキャンセルを要求...")
    for task in tasks:
        if task.done() is False:
            task.cancel()

    # キャンセルとマークされる次回のステップを待つ必要がある
    await asyncio.gather(*tasks, return_exceptions=True)

    # main() コルーチンを動かしている Task を除いて未完了な Task を集める
    pendings = [t.get_name() for t in asyncio.all_tasks() if t != asyncio.current_task()]
    print(f"{pendings=}")  # long task がキャンセルされたことがわかる


asyncio.run(main())

ZeroDivisionError: division by zero
pendings=['long task']
未完了な Task のキャンセルを要求...
キャンセルされた場合の処理を実行中...
pendings=[]


このコードと出力からわかるように、 `ZeroDivisionError` が発生した後も、 `long task` は未完了なまま残っている。 `asyncio.gather()` で未完了な `Task` オブジェクトをキャンセルする場合、 `cancel()` メソッドを呼び出す処理、そしてキャンセルとマークされるまで待つ処理の実装が必要となる（`asyncio.wait()` でも同様）。

### 非同期コンテキストマネージャー ###

**非同期コンテキストマネージャー**（asynchronous context manager）と async with 文は、それぞれコンテキストマネージャーと with 文の非同期版である。

非同期コンテキストマネージャーは、次の二つのメソッドをサポートする。

  * `__aenter__()`: 意味的には `__enter__()` と似ているが、コルーチン関数とする。
  * `__aexit__()`: 意味的には `__exit__()` と似ているが、コルーチン関数とする。

async with 文は、with 文と似ているが、次の 2 点で異なる。

  * async with 文は、コルーチン関数内だけで使用できる特別な構文である。
  * async with 文に置けるのは、非同期コンテキストマネージャーだけである。

次のコードは、非同期コンテキストマネージャーの作成例である。

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()


class AsyncContextManager:
    async def __aenter__(self):
        print("Entering the async context")
        await asyncio.sleep(1.0)
        return self

    async def __aexit__(self, exc_type, exc_value, traceback):
        print("Exiting the async context")
        await asyncio.sleep(1.0)

    async def compute(self, x, y):
        print("Compute %s + %s ..." % (x, y))
        await asyncio.sleep(1.0)
        return x + y


async def main():
    async with AsyncContextManager() as acm:
        await acm.compute(1, 2)


asyncio.run(main())

Entering the async context
Compute 1 + 2 ...
Exiting the async context


デコレーター `@contextlib.asynccontextmanager` は、`contextlib.contextmanager()` と似ているが、非同期コンテキストマネージャーを生成する。次コードは、 `@contextlib.asynccontextmanager` の使用例であり、上記のコードと等価である。

In [ ]:
import asyncio
from contextlib import asynccontextmanager

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()


async def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    await asyncio.sleep(1.0)
    return x + y


@asynccontextmanager
async def asynccontextmanager():
    try:
        print("Entering the async context")
        await asyncio.sleep(1.0)
        yield
    finally:
        print("Exiting the async context")
        await asyncio.sleep(1.0)


async def main():
    async with asynccontextmanager():
        await compute(1, 2)


asyncio.run(main())

Entering the async context
Compute 1 + 2 ...
Exiting the async context


### TaskGroup ###

``` python
asyncio.TaskGroup()
```

これは、Python 3.11 で追加されたクラスで、 `Task` オブジェクトのグループを保持し、グループに属するすべての `Task` オブジェクトが完了するのを待つ非同期コンテキストマネージャーである。

インスタンスメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `create_task(coro, *, name=None, context=None)` | このグループに `Task` オブジェクトを作成する。引数は `asyncio.create_task()` と同じ | `Task` |

`create_task()` メソッドで作成した複数の `Task` オブジェクトは、同時に実行され async with 文のブロックを抜けるタイミングで完了している。 async with 文のブロックを抜けた後には `create_task()` メソッドを使用できなくなる。

次のコードは、`asyncio.TaskGroup()` の使用例で、 `asyncio.gather()` の使用例と等価である:

``` python
import asyncio
import time

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
# import nest_asyncio
# nest_asyncio.apply()

async def compute(x, y):
    print("Compute %s + %s ... [at %s]" % (x, y, time.strftime("%X")))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    print(f"started at {time.strftime('%X')}")
    async with asyncio.TaskGroup() as tg:
        task1 = tg.create_task(compute(x, y))
        task2 = tg.create_task(compute(x * 2, y * 2))
        task3 = tg.create_task(compute(x * 10, y * 10))
    print("%s + %s = %s" % (x, y, task1.result()))
    print("%s + %s = %s" % (x * 2, y * 2, task2.result()))
    print("%s + %s = %s" % (x * 10, y * 10, task3.result()))
    print(f"finished at {time.strftime('%X')}")

asyncio.run(print_sum(1, 2))
```

### 例外グループ ###

`asyncio.TaskGroup()` と `asyncio.wait()` や `asyncio.gather()` との違いはエラーが発生した場合の扱いにある。

  * `asyncio.TaskGroup()` は、グループに属する `Task` オブジェクトのいずれかが `asyncio.CancelledError` 以外の例外で初めて失敗すると、グループ内の残りの `Task` オブジェクトを自動的にキャンセルする。 async with 文のブロックを抜けた時点では、 `Task` オブジェクトは確実にキャンセルされている。
  * `asyncio.TaskGroup()` は、発生した例外をすべて例外グループにまとめて送出する。

``` python
ExceptionGroup(msg, excs)
BaseExceptionGroup(msg, excs)
```

Python 3.11 で追加されたこの 2 つの例外は、例外グループと呼ばれ、例外インスタンスのリスト `excs` を包含し、同時に送出できるようにする。

`excs` の中に含める例外は、型ではなくインスタンスである必要がある。 `ExceptionGroup` では `excs` に `Exception` のサブクラスのインスタンスのみを含められる。 `ExceptionGroup` のコンストラクタは、`Exception` サブクラス以外の例外のインスタンスを含む場合は `TypeError` を送出する。 `BaseExceptionGroup` では `excs` に任意の例外インスタンスを含められる。 `BaseExceptionGroup` のコンストラクタは、含まれる例外がすべて `Exception` の場合は `BaseExceptionGroup` ではなく `ExceptionGroup` を返すように自動的に選択される。

インスタンス属性:

| 引数 | 意味 |
|:---|:---|
| `message` | コンストラクタの `msg` 引数。この属性は読み込み専用である |
| `exceptions` | コンストラクタに渡された一連の `excs` に含まれる例外のタプル。この属性は読み込み専用である |

インスタンスメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `subgroup(condition)` | グループで `condition` にマッチする例外のみを含む例外グループを返す。結果が空の場合は `None` を返す。<br />`condition` は唯一の引数として例外を受け取り結果に入れるべき例外に対して `True` を返す関数か、例外ク<br />ラスのタプルか、単一の例外クラスとする | 例外グループ &#124; `None` |
| `split(condition)` | `subgroup()` と似ているが、`(match, rest)` のペアを返す。`match` は `subgroup(condition)` で `rest` は残り<br />のマッチしない部分である | `tuple` |

例外グループが送出された場合の標準エラー出力の内容は、公式チュートリアルの [複数の関連しない例外の送出と処理](https://docs.python.org/ja/3/tutorial/errors.html#raising-and-handling-multiple-unrelated-exceptions) を参照。

例外グループも例外なので、他の例外と同じように捕捉できる:

``` python
try:
    try:
        excs = [OSError('error 1'), SystemError('error 2'), RuntimeError("error 3")]
        raise BaseExceptionGroup('there were problems', excs)
    except BaseExceptionGroup as e:
        print("例外グループ: ", repr(e))
        raise e.subgroup((RuntimeError,)) from None
except BaseExceptionGroup as e:
    print("例外グループ: ", repr(e))
```

内側の try-except では例外グループを捕捉し、`subgroup()` メソッドを使って `RuntimeError` を含むサブグループを再送出している。このコードを実行すると、標準出力に以下のように印字される:

``` text
(内側)例外グループ:  ExceptionGroup('there were problems', [OSError('error 1'), SystemError('error 2'), RuntimeError('error 3')])
(外側)例外グループ:  ExceptionGroup('there were problems', [RuntimeError('error 3')])
```

Python 3.11 からは、 `except` の代わりに `except*` を使用すると、グループの中にある特定の型に一致した例外だけを選択して処理できる。

``` python
try:
    excs = [OSError("error 1"), SystemError("error 2")]
    raise BaseExceptionGroup("there were problems", excs)
except* OSError as e:
    print("There were OSErrors: ", repr(e))
except* SystemError as e:
    print("There were SystemErrors: ", repr(e))
```

このコードを実行すると、標準出力に以下のように印字される:

``` text
There were OSErrors:  ExceptionGroup('there were problems', [OSError('error 1')])
There were SystemErrors:  ExceptionGroup('there were problems', [SystemError('error 2')])
```

try-except とは大きく違う点が 2 つある。

  * 複数の except* ブロックが実行される。
  * 各 except* では、発生した例外グループのサブグループとなる例外グループが生成される（`as 変数` で変数に代入される）。含まれる例外インスタンスには、 `exceptions` 属性で得られるタプルの要素としてアクセスできる。

try ブロックで例外グループが送出され、 except* で捕捉されなかった例外は、そのサブグループとして例外グループにまとめられて送出される:

``` python
try:
    try:
        excs = [OSError("error 1"), SystemError("error 2"), RuntimeError("error 3")]
        raise BaseExceptionGroup("there were problems", excs)
    except* OSError as e:
        print("There were OSErrors: ", repr(e.exceptions))
    except* SystemError as e:
        print("There were SystemErrors: ", repr(e.exceptions))
except* RuntimeError as e:
    print("There were RuntimeErrors: ", repr(e))
```

このコードを実行すると、標準出力に以下のように印字される:

``` text
There were OSErrors:  (OSError('error 1'),)
There were SystemErrors:  (SystemError('error 2'),)
There were RuntimeErrors:  ExceptionGroup('there were problems', [RuntimeError('error 3')])
```

次のコードは、 `asyncio.TaskGroup` で同時に発生する例外を全て捕捉する例である:

``` python
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
# import nest_asyncio
# nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    try:
        async with asyncio.TaskGroup() as tg:
            tasks = [tg.create_task(foo(x)) for x in (11, 0, 12, "13", 14)]
    except* ZeroDivisionError as e:
        print("There were ZeroDivisionError:", repr(e.exceptions))
    except* TypeError as e:
        print("There were TypeError:", repr(e.exceptions))

    # main() コルーチンを動かしている Task を除いて未完了な Task を集める
    pendings = [t.get_name() for t in asyncio.all_tasks() if t != asyncio.current_task()]
    print(f"{pendings=}")  # long task がキャンセルされたことがわかる

asyncio.run(main())
```

このコードを実行すると、標準出力に以下のように印字される:

``` text
There were ZeroDivisionError: (ZeroDivisionError('division by zero'),)
There were TypeError: (TypeError("unsupported operand type(s) for /: 'int' and 'str'"),)
pendings=[]
```

このコードと出力からわかるように、 async with ブロック内でエラーが発生した場合、ブロックを抜けた時には未完了な `Task` オブジェクトはキャンセルされた状態であり、キャンセルとマークされるまで待つ処理の実装は必要ない。

### キャンセルからの保護 ###

``` python
asyncio.shield(aw)
```

`asyncio.create_task()` の代わりにこの関数を使用した場合、 awaitable オブジェクト `aw` をキャンセル から保護する。 `aw` がコルーチンだった場合、自動的に `Task` オブジェクトとしてスケジュールされる。

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    task = asyncio.shield(foo(1))
    task.cancel()
    await asyncio.sleep(1)
    # main() コルーチンを動かしている Task を除いて未完了な Task を集める
    pendings = [t for t in asyncio.all_tasks() if t != asyncio.current_task()]
    print(f"{len(pendings)=}")  # task がキャンセルされないことがわかる

asyncio.run(main())

len(pendings)=1


### タイムアウト ###

``` python
asyncio.wait_for(aw, timeout)
```

この関数は、awaitable オブジェクト `aw` が、完了するかタイムアウトになるのを待つ。タイムアウトが発生した場合、 `Task` オブジェクトはキャンセルされ、`TimeoutError` が発生する（Python 3.10 以前では `asyncio.TimeoutError` が発生する。Python 3.11 以降では `asyncio.TimeoutError` は `TimeoutError` の非推奨な別名である）。そうでない場合、 `aw` の結果を返すか、例外を発生させる。

| 引数 | 意味 |
|:---|:---|
| `aw` | `aw` がコルーチンだった場合、自動的に `Task` オブジェクトとしてスケジュールされる |
| `timeout` | `None` もしくは待つ秒数の浮動小数点数か整数を指定できる。`timeout` が None の場合、`aw` が完了するまで待つ |

この関数は、 `Task` オブジェクトが実際にキャンセルされるまで待つため、待ち時間の合計は `timeout` を超えることがある。キャンセル中に例外が発生した場合は、その例外は伝播される。

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    try:
        result = await asyncio.wait_for(foo(0.1), timeout=1)  # 10 秒かかるタスクを実行
        print(result)
    except asyncio.TimeoutError:  # Python 3.11 以降では TimeoutError でよい
        print("タスクがタイムアウトしました")
    # main() コルーチンを動かしている Task を除いて未完了な Task を集める
    pendings = [t for t in asyncio.all_tasks() if t != asyncio.current_task()]
    print(f"{len(pendings)=}")  # task がキャンセルされないことがわかる

asyncio.run(main())

タスクがタイムアウトしました
len(pendings)=0


`asyncio.shield(aw)` を使うと、タスクのキャンセルを防ぐことができる。

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    try:
        task = asyncio.create_task(foo(2))
        result = await asyncio.wait_for(asyncio.shield(task), timeout=1)  # 2 秒かかるタスクを実行
        print(f"{result=}")  # タイムアウトしない
    except asyncio.TimeoutError:  # Python 3.11 以降では TimeoutError でよい
        print("タスクがタイムアウトしました")
    # main() コルーチンを動かしている Task を除いて未完了な Task を集める
    pendings = [t for t in asyncio.all_tasks() if t != asyncio.current_task()]
    print(f"{len(pendings)=}")  # task が完了した

asyncio.run(main())

result=2
len(pendings)=0


``` python
asyncio.timeout(delay)
```

これは、 Python 3.11 で追加された関数で、 `Task` オブジェクトをタイムアウト付きで実行するための非同期コンテキストマネージャーである `asyncio.Timeout` オブジェクトを返す。 `delay` 引数の意味は、 `asyncio.wait_for()` 関数の `timeout` 引数と同様である。

`asyncio.Timeout` は、async with ブロック内の await 式に置いた `Task` オブジェクトが `delay` 以上経過しても完了しなかった場合、その `Task` オブジェクトをキャンセルするよう要求し、実際にキャンセルされてから `TimeoutError` 例外を送出する。

``` python
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
# import nest_asyncio
# nest_asyncio.apply()

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    task1 = asyncio.create_task(foo(0.1), name="long task")  # 10 秒かかるタスク
    task2 = asyncio.create_task(foo(10), name="normal task")  # 0.1 秒かかるタスク
    try:
        async with asyncio.timeout(1):
            result1 = await task1
            result2 = await task2
        print(result1, result2)
    except TimeoutError:
        print("タスクがタイムアウトしました")
    # main() コルーチンを動かしている Task を除いて未完了な Task を集める
    pendings = [t.get_name() for t in asyncio.all_tasks() if t != asyncio.current_task()]
    print(f"{pendings=}")  # long task がキャンセルされたことがわかる

asyncio.run(main())
```

このコードを実行すると、標準出力に以下のように印字される:

``` text
タスクがタイムアウトしました
pendings=[]
```

なお、`asyncio.Timeout` は、複数の `Task` オブジェクトで同時に発生したエラーを例外グループで送出したり、キャンセルする機能はない。

### コンテキスト変数 ###

**コンテキスト変数**は、スレッドローカルデータの非同期版である。スレッド上のどのコルーチンもコンテキスト変数にアクセスできるが、アクセスされるデータはそのコルーチンごとに固有となるようにできる。標準ライブラリの `contextvars` モジュールは、コンテキスト変数をサポートする。

スレッドは、`contextvars.Context` オブジェクトをスレッドローカルデータとして保持する。コンテキスト変数は、 `Context` オブジェクトの中で値を持つ。 `Context` オブジェクトは、任意のタイミングでスナップショットを作成することができる。

コンテキスト変数は、 `contextvars.ContextVar` インスタンスとして、**モジュールレベルでグローバル変数として作成する必要がある**。

``` python
contextvars.ContextVar(name[, *, default])
```

| 引数 | 意味 |
|:---|:---|
| `name` | 変数の名前を指定する。読み出し専用のプロパティ `name` で参照される |
| `default` | キーワード専用引数。変数の初期値を指定できる |

インスタンスメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `ContextVar.get([default])` | 現在の `Context` のコンテキスト変数の値を返す。値がなければ、`default` 引数が指定されていればその値を返し、<br />そうでなければコンストラクタ引数 `default` の値を返し、そうでなければ `LookupError` 例外を送出する | 変数の値 |
| `ContextVar.set(value)` | 現在の `Context` のコンテキスト変数に新しい値 `value` を設定する。`contextvars.Token` オブジェクトを返す。<br />`contextvars.Token` オブジェクトは、`ContextVar.reset()` メソッドに渡すことで、対応する `set` を呼び出す前の<br />コンテキスト変数の値に戻せる | `Token` |
| `ContextVar.reset(token)` | コンテキスト変数の値を、`token` を生成した `set()` が呼び出される前の値にリセットする | `None` |

In [ ]:
import contextvars

var = contextvars.ContextVar("var", default=42)
assert var.get() == 42
token = var.set(1)
try:
    assert var.get() == 1
finally:
    var.reset(token)
assert var.get() == 42

``` python
contextvars.copy_context()
```

この関数は、 `Context` オブジェクトのスナップショットを作成して返す。

`Context` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `Context.run(callable, *args, **kwargs)` | `callable(*args, **kwargs)` を `run` メソッドが呼ばれた `Context` の中で実行する。実<br />行した結果を返すか、例外が発生した場合はその例外を伝播する | `callable()` の結果 |
| `context[var]` | コンテキスト変数 `var` の値を返す。`Context` オブジェクト内で `var` の値が設定されてい<br />ない場合は、`KeyError` を送出する | `var` の値 |
| `get(var[, default])` | `Context` オブジェクト内で `var` の値が設定されていればその値を返す。そうでなければ、<br />`default` を返す。`default` を指定していなければ、`None` を返す | `var` の値 |
| `Context.iter(context)`| `Context` オブジェクトに格納されている変数群のイテレーターを返す | イテレーター |
| `Context.len(proxy)` | `Context` オブジェクトに格納されている変数の数を返す | `int` |
| `Context.keys()` | `Context` オブジェクト中のすべての変数のリストを返す | `list` |
| `Context.values()` | `Context` オブジェクト中のすべての変数の値のリストを返す | `list` |
| `Context.items()` | `Context` オブジェクト中のすべての変数について、変数とその値からなる 2 要素のタプル<br />のリストを返す | `list` |

In [ ]:
import asyncio
import contextvars

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()
number_var = contextvars.ContextVar("number_var")

async def foo(x):
    await asyncio.sleep(1 / x)
    n = number_var.get()
    print(f"foo: number={n}")  # foo: number=1
    n += 1
    number_var.set(n)

async def main():
    n = 1
    number_var.set(n)
    print(f"main: number={n}")  # main: number=1
    ctx = contextvars.copy_context()
    await ctx.run(foo, 10)
    n = number_var.get()
    print(f"main: number={n}")  # main: number=2
    print(f"main: original number={ctx[number_var]}")  # main: original number=1

asyncio.run(main())

main: number=1
foo: number=1
main: number=2
main: original number=1


`asyncio.Runner` インスタンスの `run()` メソッド、 `asyncio.create_task()` 関数、 `asyncio.TaskGroup` インスタンスの `create_task()` メソッドでは、キーワード専用引数 `context` に `Context` オブジェクトのスナップショットを渡すことができる。 `context` がデフォルトの `None` の場合、各関数では内部で `Context` オブジェクトのスナップショットが作成される。各関数でスケジュールされたコールバックは、 `Context` オブジェクトの `run()` メソッドで実行される。

### スレッド内での実行 ###

Python 3.9 以降では、 `asyncio` の非同期処理と `concurrent.futures.ThreadPoolExecutor` を簡単に共存させることができる。

``` python
asyncio.to_thread(func, /, *args, **kwargs)
```

この関数は、内部で `concurrent.futures.ThreadPoolExecutor` を利用して、別のスレッドで非同期的に関数 `func` を実行する。この関数に渡された `*args` と `**kwargs` は関数 `func` に直接渡される。また、現在の `contextvars.Context` も伝播される。この関数は、関数 `func` の最終結果を待ち受けできるコルーチンを返す。

次のコードは、`asyncio.to_thread()` の使用例である。 `blocking_io()` 関数は、ブロッキング I/O の待機時間の長い操作を行う関数を表現している。ブロッキング I/O は、 `asyncio` に対応できないから別のスレッドで処理させている。

In [ ]:
import asyncio
import time
# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

def blocking_io():
    print(f"start blocking_io at {time.strftime('%X')}")
    time.sleep(1)
    print(f"blocking_io complete at {time.strftime('%X')}")

async def foo(x):
    await asyncio.sleep(1 / x)
    return x

async def main():
    print(f"started main at {time.strftime('%X')}")
    await asyncio.gather(
        asyncio.to_thread(blocking_io),
        foo(10),
    )
    print(f"finished main at {time.strftime('%X')}")

asyncio.run(main())

started main at 05:41:25
start blocking_io at 05:41:25
blocking_io complete at 05:41:26
finished main at 05:41:26


### 同期プリミティブ ###

`asyncio` は、 `threading` モジュールの同期プリミティブと似た同期プリミティブを提供する。それらは、 async/await コードから使われるために特別に設計されている。ただし、 `asyncio` 同期プリミティブのメソッドは `timeout` 引数を持たない。

以下の 4 つの `asyncio` 同期プリミティブは、非同期コンテキストマネージャーであり、 async with 文と組み合わせて使うことが推奨される。

  * `asyncio.Lock`
  * `asyncio.Semaphore`
  * `asyncio.BoundedSemaphore`
  * `asyncio.Condition`

次のコード片

``` python
lock = asyncio.Lock()

# ... later
async with lock:
    # access shared state
```

これは、以下と同じ。

``` python
lock = asyncio.Lock()

# ... later
await lock.acquire()
try:
    # access shared state
finally:
    lock.release()
```

`asyncio.Event` は、複数のタスクに対して何らかのイベントが発生したことを通知するために使うことができる。`wait()` メソッドはコルーチン関数になっている。以下は、 `asyncio.Event` の使用例である。

In [ ]:
import asyncio

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def worker(event):
    task = asyncio.current_task()
    print(f"{task.get_name()}: start")
    await event.wait()
    print(f"{task.get_name()}: end")

async def event_trigger(event):
    task = asyncio.current_task()
    print(f"{task.get_name()}: start")
    await asyncio.sleep(1.0)
    print(f"{task.get_name()}: end")
    event.set()

async def main():
    event = asyncio.Event()
    await asyncio.gather(
        asyncio.create_task(worker(event), name="worker"),
        asyncio.create_task(event_trigger(event), name="event_trigger"),
    )

asyncio.run(main())

worker: start
event_trigger: start
event_trigger: end
worker: end


`asyncio.Barrier` は Python 3.11 で追加された。 `threading.Barrier` と異なり、コンストラクタは `action` 引数も `timeout` も受け付けない。 `wait()` メソッドは、タスクが待っている間にバリアが破壊された場合、`asyncio.BrokenBarrierError` 例外を送出する。公式ドキュメントの[使用例](https://docs.python.org/ja/3/library/asyncio-sync.html#asyncio-example-barrier)を参照。

### キュー ###

`asyncio` は、 `queue` モジュールのキューと似たキューを提供する。

  * `asyncio.Queue`
  * `asyncio.LifoQueue`
  * `asyncio.PriorityQueue`

これらは、 async/await コードから使われるために特別に設計されている。ただし、 `asyncio` キューのメソッドは `block` 引数と `timeout` 引数を持たない。主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `put(item)` | コルーチン関数。`item` をキューに挿入し、「未完了タスクのカウント」をインクリメントする。キューの上限が設定されていて上限に<br />達していた場合、挿入処理はキューのアイテムが取り出されて空きが出るまでブロックされる | `None` |
| `put_nowait(item)` | ブロックせずにアイテムをキューに追加する。上限までの空きがない場合、`asyncio.QueueFull` 例外を送出する | `None` |
| `get()` | コルーチン関数。キューからアイテムを取り出す。キューが空の場合アイテムが入るまで待機する | アイテム |
| `get_nowait()` | キューが空でなければ直ちにアイテムを取り出す。そうでなければ `asyncio.QueueEmpty` を返す | アイテム |
| `empty()` | キューが空の場合は `True` を返し、そうでなければ `False` を返す | `bool` |
| `full()` | キューが一杯の場合は `True` を返し、そうでなければ `False` を返す | `bool` |
| `task_done()` | 内部イベントの `set()` を呼び出し、`get()` で待機中の全てのタスクを起こす。また、「未完了タスクのカウント」をデクリメントする。<br />この結果 0 未満になった場合（つまりキューにある要素より多く呼び出された場合） `ValueError` 例外が発生する | `None` |
| `join()` | コルーチン関数。「未完了タスクのカウント」が 0 になるまで待機する | `None` |
| `qsize()` | キュー内のアイテム数を返す | `int` |

次のコードは、5 つのアイテム（1 から 5 までの整数）の処理を 2 つの `Task` オブジェクトで同時に行うために `asyncio.Queue` を使用する例である。スレッドプールの非同期版を実装している。

In [ ]:
import asyncio
import time

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()


async def worker(q):
    # キューコンシューマー
    while not q.empty():
        item = await q.get()
        task = asyncio.current_task()
        print(f"{task.get_name()}: Working on {item}")
        await asyncio.sleep(item * 0.1)
        print(f"{task.get_name()}: Finished {item}")
        q.task_done()


async def main():
    # Queue を設定
    q = asyncio.Queue()
    total_sleep_time = 0
    for i in range(1, 6):
        total_sleep_time += i * 0.1
        q.put_nowait(i)

    # ワーカータスクのプールを設定（2 つの Task オブジェクトからなる）
    tasks = []
    for i in range(2):
        task = asyncio.create_task(worker(q), name=f"t{i}")
        tasks.append(task)

    started_at = time.time()
    # キューが空になるまで待機
    await q.join()
    total_slept_for = time.time() - started_at

    # タスクはもう用済みなのでキャンセルする
    for task in tasks:
        task.cancel()
    # すべてのワーカータスクがキャンセルされるまで待機する
    await asyncio.gather(*tasks, return_exceptions=True)

    print("====")
    print(f"2 workers slept in parallel for {total_slept_for:.2f} seconds")
    print(f"total expected sleep time: {total_sleep_time:.2f} seconds")


asyncio.run(main())

t0: Working on 1
t1: Working on 2
t0: Finished 1
t0: Working on 3
t1: Finished 2
t1: Working on 4
t0: Finished 3
t0: Working on 5
t1: Finished 4
t0: Finished 5
====
2 workers slept in parallel for 0.91 seconds
total expected sleep time: 1.50 seconds


### ストリーム ###

`asyncio` の非同期プロセス間通信は、以下のような仕組みになっている。

まず、 `asyncio` イベントループは、 `socket` モジュールと同等の API を備えており、ノンブロッキングなソケットを作成し、これとコールバックの組み合わせを I/O 多重化実装に登録し、さらにリモートソケットへの接続とデータの送受信を行うことができる。

これらの機能は、トランスポートおよびプロトコルとして抽象化されている。プロトコルは、通信プロトコルを実装し、データを送信するためにトランスポートのメソッドを呼び出す。トランスポートはソケットの抽象化であり、TCP、SSL、サブプロセスパイプなどを実装し、受信したデータを渡すためにプロトコルのメソッドを呼び出す。

プロトコルは、トランスポートの種類ごとに抽象基底クラスが用意されている。つまり、プロトコルを選ぶことで、使用されるトランスポートが決まる。たとえば、`asyncio.Protocol` クラスは、ストリームソケット（TCP、Unix ソケット）のためのトランスポートを用いたプロトコルを表現する抽象基底クラスである。具体的な実装は、サードパーティ製のサブクラスにおいて行われることになる。 `asyncio` 自らも `asyncio.Protocol` の具象クラスを用意し、その上に構築された高レベルなストリーム API を提供している。

``` python
asyncio.open_connection(host=None, port=None, *, limit=_DEFAULT_LIMIT, **kwds)
```

これはコルーチン関数であり、ネットワーク接続を確立し、`(reader, writer)` の 2 要素タプルを返す。主な引数は以下のとおり。

| 引数 | 意味 |
|:---|:---|
| `host` | リモートソケットのアドレス |
| `port` | ポート番号 |
| `limit` | `reader` オブジェクトが利用するバッファのサイズの上限値。デフォルトでは 64 KiB に設定される |
| `ssl` | `None`（デフォルト）の場合、暗号化なしの TCP トランスポートが作成される。`True` の場合、SSL/TLS トランスポートが作成される |
| `family` | アドレスファミリ。`socket` モジュール定数に従った整数を指定する |

戻り値の第 1 要素 `reader` は、 `asyncio.StreamReader` のインスタンスである。主なメソッドは以下のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `StreamReader.read(n=-1)` | コルーチン関数。ストリームから最大 `n` バイト読み込み `bytes` オブジェクトを返す。1 バイトも読み込まないうちに EOF を<br />受信したなら、空のバイト列を返す。`n` が 0 なら、ただちに空のバイト列を返す。`n` が `-1`（デフォルト）の場合、EOF になる<br />まで読み込み、読み込まれた全てのバイト列を返す | `bytes` |
| `StreamReader.readline()` | コルーチン関数。1 行（`\n` で終了するバイト列のシーケンス）読み込む | `bytes` |

戻り値の第 2 要素 `writer` は、 `asyncio.StreamWriter` のインスタンスである。主なメソッドは以下のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `StreamWriter.write(data)` | 背後にあるソケット `data` を即座に書き込む。書き込みに失敗した場合、データは送信可<br />能になるまで内部の書き込みバッファーに格納されて待機する。このメソッドは `drain()` <br />メソッドと組み合わせて使う | `None` |
| `StreamWriter.writelines(data)` | `data` にバイト列のリストを受け付けること以外は `write()` と同じ | `None` |
| `StreamWriter.drain()` | コルーチン関数。ストリームへの書き込み再開に適切な状態になるまで待つ | `None` |
| `StreamWriter.close()` | ストリームと背後にあるソケットを閉じる。このメソッドは `wait_closed()` メソッドと組み<br />合わせて使う | `None` |
| `StreamWriter.wait_closed()` | コルーチン関数。ストリームが閉じるまで待機する `None` |
| `StreamWriter.get_extra_info(name, default=None)` | トランスポートを返す | トランスポート |

``` python
asyncio.start_server(client_connected_cb, host=None, port=None, *, limit=_DEFAULT_LIMIT, **kwds):
```

これはコルーチン関数であり、ソケットサーバーを起動する。`client_connected_cb` コールバックは新しいクライアント接続が確立されるたびに呼び出される。このコールバックは `StreamReader` と `StreamWriter` クラスのインスタンスの組 `(reader, writer)` を 2 つの引数として受け取る。

``` python
asyncio.open_unix_connection(path=None, *, limit=_DEFAULT_LIMIT, **kwds)
```

これはコルーチン関数であり、Unix ソケット接続を確立し、`(reader, writer)` のオブジェクトのペアを返す。

``` python
asyncio.start_unix_server(client_connected_cb, path=None, *, limit=_DEFAULT_LIMIT, **kwds)
```

これはコルーチン関数であり、Unix ソケットサーバーを起動する。

次のコードは、ブロッキング I/O のコード例の `request_get()` を非同期処理に対応するように書き直したものである。

In [ ]:
import asyncio
import time

# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()

async def request_get(i):
    start = time.time()
    response = b""
    reader, writer = await asyncio.open_connection("example.com", 80)
    writer.write(b"GET / HTTP/1.0\r\nHost: example.com\r\n\r\n")
    recv = await reader.readline()
    while recv:
        response += recv
        recv = await reader.readline()
    writer.close()
    await writer.wait_closed()
    print("{} 処理時間: {:.3f} 秒".format(i, time.time() - start))
    return response

async def main():
    print("計測開始")
    startall = time.time()
    tasks = []
    for i in range(10):
        task = asyncio.create_task(request_get(i))
        tasks.append(task)
    await asyncio.wait(tasks)
    print("{:.3f} 秒経過した".format(time.time() - startall))

asyncio.run(main())

計測開始
2 処理時間: 0.033 秒
3 処理時間: 0.034 秒
5 処理時間: 0.035 秒
4 処理時間: 0.036 秒
7 処理時間: 0.037 秒
1 処理時間: 0.062 秒
6 処理時間: 0.062 秒
0 処理時間: 0.064 秒
8 処理時間: 0.065 秒
9 処理時間: 0.070 秒
0.071 秒経過した


`asyncio` では、非同期 I/O に基づくプログラムが、コールバックのネストを使用することなく、ブロッキング I/O のコードと似た形で書けることがわかる。

aiohttp
-------

サードパーティ製 [aiohttp](https://docs.aiohttp.org/en/stable/) パッケージは、 `asyncio` に対応する非同期 HTTP Client/Server 用のフレームワークである。ライセンスは Apache license 2.0。インストール方法は次のとおり。

``` shell
pip install aiohttp
```

`aiohttp.ClientSession` クラスは、 HTTP クライアントのセッションを待つ非同期コンテキストマネージャーである。そのインスタンスは、 `request()`、 `get()`、 `post()`、 `put()`、 `patch()`、 `delete()`、 `head()` メソッドを持つ。これらのリクエストメソッドは、 Requests パッケージで提供される同名の関数の非同期コンテキストマネージャー版である。戻り値は `aiohttp.ClientResponse` インスタンスである。

`aiohttp.ClientResponse` クラスは、 Requests パッケージの `Response` クラスに似ているが、属性やメソッドの名前が微妙に異なることに注意する。非同期コンテキストマネージャープロトコルをサポートする:

``` python
resp = await client_session.get(url)
async with resp:
    assert resp.status == 200
```

async with 文のブロックを抜けると、接続が解放される。

| 属性 | 意味 |
|:---|:---|
| `Response.url` | リクエストした URL |
| `Response.cookies` | レスポンスに含まれる Cookie 情報を保持するオブジェクト |
| `Response.headers` | レスポンスヘッダーの内容を表す辞書（CIMultiDictProxy） |
| `Response.status` | レスポンスの HTTP ステータスコード |
| `Response.charset` | （読み出し専用）エンコーディング |
| `Response.ok` | （読み出し専用）レスポンスの HTTP ステータスコードが 400 より小さいならば `True`、そうでない場合は `False` |
| `Response.content` | `asyncio.StreamReader` のインスタンス |

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `Response.raise_for_status()` | レスポンスの HTTP ステータスコードが 400 番台や 500 番台だった場合に、`aiohttp.ClientResponseError` 例<br />外を送出する | `None` |
| `Response.read()` | コルーチン関数。レスポンスの本文全体をバイト列として読み取る。データの読み取りでエラーが発生した場合は<br />基礎となる接続を閉じ、そうでない場合は接続を解放する。データを読み取れない場合は、<br />`aiohttp.ClientResponseError` を送出する | `bytes` |
| `Response.text(encoding=None)` | コルーチン関数。レスポンスの本文を読み取り、指定されたエンコードを使用してデコードされた文字列を返す。<br />`encoding` が `None`（デフォルト）の場合、エンコードは Content-Type ヘッダーから、または<br /> `fallback_charset_resolver()` 関数を使用して決定される | `str` |
| `Response.json(*, encoding=None,`<br />` loads=json.loads,`<br />` content_type='application/json')` | コルーチン関数。レスポンス本体を JSON 形式として解析し、辞書に変換して返す。変換には `encoding` と `loads`<br /> が使用される。`content_type` に指定したレスポンスのコンテンツタイプと一致しない場合、<br />`aiohttp.ContentTypeError` が発生する。`content_type` を `None` に指定すると、このチェックを無効にする | `dict` |

In [ ]:
import asyncio
import time

import aiohttp
# Colab 上で実行するのでなけれは以下の2行をコメントアウトする
import nest_asyncio
nest_asyncio.apply()


async def request_get(session, i):
    start = time.time()
    async with session.get("http://example.com") as resp:
        response = await resp.read()
    print("{} 処理時間: {:.3f} 秒".format(i, time.time() - start))
    return response


async def main():
    print("計測開始")
    startall = time.time()
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(10):
            task = asyncio.create_task(request_get(session, i))
            tasks.append(task)
        await asyncio.wait(tasks)
    print("{:.3f} 秒経過した".format(time.time() - startall))


asyncio.run(main())

計測開始
1 処理時間: 0.031 秒
3 処理時間: 0.030 秒
8 処理時間: 0.031 秒
2 処理時間: 0.034 秒
7 処理時間: 0.033 秒
6 処理時間: 0.034 秒
9 処理時間: 0.033 秒
5 処理時間: 0.034 秒
0 処理時間: 0.039 秒
4 処理時間: 0.038 秒
0.045 秒経過した


psutil
------

サードパーティ製の [psutil](https://psutil.readthedocs.io/en/latest/) は、Python で実行中のプロセスとシステム（CPU、メモリ、ディスク、ネットワークなど）に関する情報を取得するためのクロスプラットフォームなライブラリである。ライセンスは BSD 3-clause License。インストール方法は次のとおり。

``` shell
pip install psutil
```

### CPU 情報 ###

``` python
psutil.cpu_count(logical=True)
```

CPU コア数を返す。`logical` 引数が `True` の場合は、`os.cpu_count()` と同様に論理コア数を返す。`logical` 引数が `False` の場合は、物理コア数を返す。

In [ ]:
import psutil
# CPU の物理コア数
psutil.cpu_count(logical=False)

1

``` python
psutil.cpu_percent(interval=None, percpu=False)
```

現在の CPU 使用率（%）を返す。 `interval` 引数を `0.0` より大きい数値で設定した場合、その `interval` 秒の間で使用率を計測する。 `percpu` 引数が `True` の場合、コアごとの使用率をリストで返す。

In [ ]:
import psutil
# CPU 使用率
psutil.cpu_percent(interval=1)

13.0

``` python
psutil.cpu_freq(percpu=False)
```

CPU 周波数を、Mhz 単位で表される現在の周波数、最小周波数、最大周波数を含む名前付きタプルとして返す。 `percpu` 引数が `True` の場合、コアごとの使用率で名前付きタプルのリストで返す。

In [ ]:
import psutil
# CPU 周波数
psutil.cpu_freq()

scpufreq(current=2200.182, min=0.0, max=0.0)

``` python
psutil.getloadavg()
```

過去 1 分間、5 分間、15 分間の CPU 待ちタスク数（実行中のタスクおよび待機中のタスクの平均数）をタプルとして返す。

In [ ]:
import psutil
# CPU 待ちタスク数
psutil.getloadavg()

(1.44775390625, 0.68017578125, 0.26171875)

### メモリ情報 ###

``` python
psutil.virtual_memory()
```

メモリの使用状況に関する統計を、バイト単位で表現された次のフィールドを含む名前付きタプルとして返す。

| フィールド | 意味 |
|:---|:---|
| `total` | 物理メモリの合計 |
| `available` | 利用可能な物理メモリの空き容量 |
| `percent` | メモリ使用率 `(total - available) / total * 100` |
| `used` | 使用済みメモリ |
| `free` | 未使用メモリ（Windows では `available` と同じ） |
| `active` | (UNIX) 現在使用中またはごく最近使用されたメモリ |
| `inactive` | (UNIX) 未使用としてマークされているメモリ |
| `buffers` | (Linux、BSD) ファイルシステム用のキャッシュ |
| `cached` | (Linux、BSD) さまざまなもののキャッシュ |
| `shared` | (Linux、BSD) 複数のプロセスが同時にアクセスできる共有メモリ |
| `slab` | (Linux) カーネル内データ構造キャッシュ |
| `wired` | (BSD、macOS) 常に RAM 内に留まるようにマークされたメモリ（ディスクに移動されない） |

In [ ]:
import psutil
# メモリの使用状況
psutil.virtual_memory()

svmem(total=13609431040, available=12449140736, percent=8.5, used=825339904, free=7721136128, active=681377792, inactive=4848025600, buffers=385798144, cached=4677156864, shared=1724416, slab=226885632)

``` python
psutil.swap_memory()
```

スワップメモリの統計情報を、次のフィールドを含む名前付きタプルとして返す。

| フィールド | 意味 |
|:---|:---|
| `total` | スワップメモリの合計（バイト単位） |
| `used` | 使用済みスワップメモリ（バイト単位） |
| `free` | 空きスワップメモリ（バイト単位） |
| `percent` | スワップメモリ使用率 `(total - available) / total * 100` |
| `sin` | システムがディスクからスワップインしたバイト数（累積）。Windows では常に 0 |
| `sout` | システムがディスクからスワップアウトしたバイト数（累積）。Windows では常に 0 |

In [ ]:
import psutil
# スワップメモリの統計情報
psutil.swap_memory()

sswap(total=0, used=0, free=0, percent=0.0, sin=0, sout=0)

### ディスク情報 ###

``` python
psutil.disk_partitions(all=False)
```

UNIX の `df` コマンドと同様に、マウントされたすべてのディスクのパーティションを、デバイス、マウント ポイント、ファイルシステムタイプを含む名前付きタプルのリストとして返す。 `all` 引数が `False`（デフォルト）の場合、物理デバイス（ハードディスク、CD-ROM ドライブ、USB キーなど）についての情報を返し、仮想デバイスは無視する（BSD ではこのパラメータは無視される）。次のフィールドを持つ名前付きタプルのリストを返す。

| フィールド | 意味 |
|:---|:---|
| `device` | デバイスのパス（例: `'/dev/hda1'`）。Windows では、これはドライブ文字（例: `'C:\\'`） |
| `mountpoint` | マウントポイントのパス（例: `'/'`）。Windows では、これはドライブ文字（例: `'C:\\'`） |
| `fstype` | パーティションのファイルシステム（例: UNIX では `'ext3'`、Windows では`'NTFS'`） |
| `opts` | ドライブ/パーティションのさまざまなマウントオプションを示すカンマ区切りの文字列 |

In [ ]:
import psutil
# パーティション情報
psutil.disk_partitions()

[sdiskpart(device='/dev/root', mountpoint='/usr/sbin/docker-init', fstype='ext2', opts='ro,relatime', maxfile=255, maxpath=4096),
 sdiskpart(device='/dev/sda1', mountpoint='/kaggle/input', fstype='ext4', opts='ro,relatime,commit=30', maxfile=255, maxpath=4096),
 sdiskpart(device='/dev/sda1', mountpoint='/etc/resolv.conf', fstype='ext4', opts='rw,nosuid,nodev,relatime,commit=30', maxfile=255, maxpath=4096),
 sdiskpart(device='/dev/sda1', mountpoint='/etc/hostname', fstype='ext4', opts='rw,nosuid,nodev,relatime,commit=30', maxfile=255, maxpath=4096),
 sdiskpart(device='/dev/sda1', mountpoint='/etc/hosts', fstype='ext4', opts='rw,nosuid,nodev,relatime,commit=30', maxfile=255, maxpath=4096)]

``` python
psutil.disk_usage(path)
```

`path` があるパーティションのディスク使用状況統計を、バイト単位で表された合計、使用済み、空き容量、および使用率を含む名前付きタプルとして返す。パスが存在しない場合は `OSError` 例外が発生する。実は、標準ライブラリの `shutil` モジュールにも同じ関数 `shutil.disk_usage()` が含まれている。

In [ ]:
import psutil
# ディスク使用状況統計
psutil.disk_usage('/')

sdiskusage(total=115658190848, used=29351915520, free=86289498112, percent=25.4)

``` python
psutil.disk_io_counters(perdisk=False, nowrap=True)
```

システム全体のディスク I/O 統計を、次のフィールドを含む名前付きタプルとして返す。

| フィールド | 意味 |
|:---|:---|
| `read_count` | 読み取り回数 |
| `write_count` | 書き込み回数 |
| `read_bytes` | 読み取りバイト数 |
| `write_bytes` | 書き込みバイト数 |
| `read_time` | (NetBSD と OpenBSD を除くすべて) ディスクからの読み取りにかかった時間（ミリ秒単位） |
| `write_time` | (NetBSD と OpenBSD を除くすべて) ディスクへの書き込みにかかった時間（ミリ秒単位） |
| `busy_time` | (Linux、FreeBSD) 実際の I/O の実行にかかった時間（ミリ秒単位） |
| `read_merged_count` | (Linux) マージされた読み取り回数 |
| `write_merged_count` | (Linux) マージされた書き込み回数 |

`perdisk` 引数が `True` の場合、システムにインストールされているすべての物理ディスクについて、パーティション名をキーとし、上記の名前付きタプルを値とする辞書として同じ情報を返す。

`nowrap` 引数が `True` の場合、`psutil` は関数呼び出し全体でそれらの数値を検出して調整し、「古い値」を「新しい値」に追加して、返される数値が常に増加するか同じままになり、減少しないようにする。

Windows では、IO カウンターを有効にするために、最初に `cmd.exe` から `diskperf -y` コマンドを発行する必要がある場合がある。

In [ ]:
import psutil
# ディスク I/O 統計
psutil.disk_io_counters()

sdiskio(read_count=193469, write_count=88903, read_bytes=4735588352, write_bytes=1846543360, read_time=281048, write_time=185128, read_merged_count=4221, write_merged_count=43732, busy_time=98465)

### ネットワーク情報 ###

``` python
psutil.net_io_counters(pernic=False, nowrap=True)
```

システム全体のネットワーク I/O 統計を、次のフィールドを含む名前付きタプルとして返す。

| フィールド | 意味 |
|:---|:---|
| `bytes_sent` | 送信されたバイト数 |
| `bytes_recv` | 受信されたバイト数 |
| `packets_sent` | 送信されたパケット数 |
| `packets_recv` | 受信されたパケット数 |
| `errin` | 受信中のエラーの合計数 |
| `errout` | 送信中のエラーの合計数 |
| `dropin` | ドロップされた受信パケットの合計数 |
| `dropout` | ドロップされた送信パケットの合計数（macOS および BSD では常に 0） |

`pernic` が `True` の場合、システムにインストールされているすべてのネットワーク・インターフェイスについて、ネットワーク・インターフェイス名をキーとし、上記の名前付きタプルを値とする辞書として同じ情報を返す。

`nowrap` が `True` の場合、`psutil` は関数呼び出し全体でこれらの数値を検出して調整し、「古い値」を「新しい値」に追加して、返される数値が常に増加するか同じままになり、減少しないようにする。

In [ ]:
import psutil
# ネットワーク I/O 統計
psutil.net_io_counters()

snetio(bytes_sent=1163729, bytes_recv=1326328, packets_sent=3695, packets_recv=3783, errin=0, errout=0, dropin=0, dropout=0)

``` python
psutil.net_connections(kind='inet')
```

システム全体のソケット接続を、次のフィールドを含む名前付きタプルのリストとして返す。

| フィールド | 意味 |
|:---|:---|
| `fd` | ソケットのファイルディスクリプタ。Windows および SunOS では、これは常に -1 |
| `family` | アドレスファミリ。`AF_INET`、`AF_INET6`、または `AF_UNIX` のいずれか |
| `type` | アドレスタイプ。`SOCK_STREAM`、`SOCK_DGRAM`、または `SOCK_SEQPACKET` のいずれか |
| `laddr` | ローカルアドレス。`(ip、port)` 名前付きタプル、または `AF_UNIX` ソケットの場合はパス |
| `raddr` | リモートアドレス。`(ip、port)` 名前付きタプル、または UNIX ソケットの場合は絶対パス |
| `status` | TCP 接続のステータス。UDP および UNIX ソケットの場合、これは常に `psutil.CONN_NONE` になる |
| `pid` | ソケットを開いたプロセスの PID。PID を取得できない場合は `None` |

`kind` 引数は、次の接続の種類を表す文字列を指定する。

| `kind` | 接続 |
|:---|:---|
| `'inet'` | IPv4 および IPv6 |
| `'inet4'` | IPv4 |
| `'inet6'` | IPv6 |
| `'tcp'` | TCP |
| `'tcp4'` | TCP over IPv4 |
| `'tcp6'` | TCP over IPv6 |
| `'udp'` | UDP |
| `'udp4'` | UDP over IPv4 |
| `'udp6'` | UDP over IPv6 |
| `'unix'` | UNIX socket (UDP と TCP プロトコルの両方) |
| `'all'` | 全て |

In [ ]:
import psutil
# ソケット接続情報
psutil.net_connections()

[sconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='172.28.0.12', port=6000), raddr=addr(ip='172.28.0.12', port=54126), status='TIME_WAIT', pid=None),
 sconn(fd=3, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='127.0.0.1', port=3453), raddr=(), status='LISTEN', pid=62),
 sconn(fd=8, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='172.28.0.12', port=9000), raddr=addr(ip='172.28.0.12', port=33836), status='ESTABLISHED', pid=84),
 sconn(fd=8, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='127.0.0.1', port=54096), raddr=addr(ip='127.0.0.1', port=43275), status='ESTABLISHED', pid=85),
 sconn(fd=-1, family=<AddressFamily.AF_INET6: 10>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='::ffff:172.28.0.12', port=8080), raddr=addr(ip='::ffff:172.28.0.1', port=41894), status='TIME_WAIT', pid=None),
 sconn(fd=3, family=<AddressFamily.AF_INET: 2

``` python
psutil.net_if_addrs()
```

システムにインストールされている各 NIC（ネットワークインターフェイスカード）に関連付けられたアドレスを辞書として返す。辞書のキーは NIC 名で、値は NIC に割り当てられた各アドレスの名前付きタプルのリストとなる。各名前付きタプルには次のフィールドが含まれる。

| フィールド | 意味 |
|:---|:---|
| `family` | アドレスファミリ。`AF_INET`、`AF_INET6` または `psutil.AF_LINK` のいずれかで、MAC アドレスを参照する |
| `address` | プライマリ NIC アドレス |
| `netmask` | ネットマスクアドレス。`None` の場合もある |
| `broadcast` | ブロードキャストアドレス。`None` の場合もある |
| `ptp` | VPN 上の宛先アドレス（ポイントツーポイント）。`None` の場合もある |

In [ ]:
import psutil
# NIC 情報
psutil.net_if_addrs()

{'lo': [snicaddr(family=<AddressFamily.AF_INET: 2>, address='127.0.0.1', netmask='255.0.0.0', broadcast=None, ptp=None),
  snicaddr(family=<AddressFamily.AF_PACKET: 17>, address='00:00:00:00:00:00', netmask=None, broadcast=None, ptp=None)],
 'eth0': [snicaddr(family=<AddressFamily.AF_INET: 2>, address='172.28.0.12', netmask='255.255.0.0', broadcast='172.28.255.255', ptp=None),
  snicaddr(family=<AddressFamily.AF_PACKET: 17>, address='02:42:ac:1c:00:0c', netmask=None, broadcast='ff:ff:ff:ff:ff:ff', ptp=None)]}